<a href="https://colab.research.google.com/github/ll3i/FashionData/blob/main/%EC%96%B4%EC%85%88%EB%B8%94_%EC%B5%9C%EC%A2%85%EC%BD%94%EB%93%9C_%EB%AF%B8%EC%85%981~3_%ED%95%A9%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 드라이브 마운트

In [ ]:
import os
import re
import json
import pandas as pd
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import numpy as np
from torchvision import models, transforms
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




---



##[1-1]

In [ ]:
import os
import pandas as pd
from collections import defaultdict

# 데이터 폴더 경로 설정
directories = {
    'training': '/content/drive/MyDrive/dataset/training_image',
    'validation': '/content/drive/MyDrive/dataset/validation_image'
}

# 각 폴더별 집계 결과를 저장할 딕셔너리
folder_results = {}

# 각 폴더에 대해 이미지 개수를 집계
for folder_name, directory in directories.items():
    # 성별 및 스타일별 이미지 개수를 저장할 구조
    image_count = defaultdict(lambda: defaultdict(int))  # 중첩된 딕셔너리 구조: {성별: {스타일: 이미지 수}}
    total_images = 0  # 전체 이미지 수를 저장할 변수

    # 파일 이름에서 성별과 스타일을 추출하여 통계 내기
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):  # JPG 파일만 처리
            parts = filename.split('_')  # 파일명을 '_'로 분할
            # 성별과 스타일 정보 추출
            gender_identifier = parts[-1][0]  # 마지막 부분의 첫 글자 (W or T)
            style = parts[3]  # 네 번째 부분은 스타일 정보
            # 성별을 '여성' / '남성'으로 변환
            gender = '여성' if gender_identifier == 'W' else '남성'
            # 성별과 스타일의 이미지 수를 집계
            image_count[gender][style] += 1
            total_images += 1  # 전체 이미지 수 증가

    # DataFrame으로 변환
    data = []
    for gender, styles in image_count.items():
        for style, count in styles.items():
            data.append([gender, style, count])

    df = pd.DataFrame(data, columns=['성별', '스타일', '이미지 수'])

    # 예쁘게 출력
    df_sorted = df.sort_values(by=['성별', '스타일']).reset_index(drop=True)

    # 폴더별 집계 결과 저장
    folder_results[folder_name] = {
        'total_images': total_images,
        'df': df_sorted
    }

# 각 폴더의 총 이미지 수 및 DataFrame 출력
for folder_name, results in folder_results.items():
    print(f"{folder_name.capitalize()} 이미지 폴더")
    print(f"총 이미지 수: {results['total_images']}")
    display(results['df'])
    print("\n" + "="*50 + "\n")


Training 이미지 폴더
총 이미지 수: 4070


,성별,스타일,이미지 수
0,남성,bold,268
1,남성,hiphop,274
2,남성,hippie,260
3,남성,ivy,237
4,남성,metrosexual,278
5,남성,mods,269
6,남성,normcore,364
7,남성,sportivecasual,298
8,여성,athleisure,67
9,여성,bodyconscious,95




Validation 이미지 폴더
총 이미지 수: 951


,성별,스타일,이미지 수
0,남성,bold,57
1,남성,hiphop,66
2,남성,hippie,82
3,남성,ivy,79
4,남성,metrosexual,58
5,남성,mods,80
6,남성,normcore,51
7,남성,sportivecasual,52
8,여성,athleisure,14
9,여성,bodyconscious,23




---



##[1-2]

In [ ]:
!pip install torch_optimizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import os
import torch_optimizer as optim  # torch_optimizer 라이브러리 사용

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 클래스 목록 (성별 + 스타일 조합으로 구성)
fashion_classes = [
    "남성_bold", "남성_hiphop", "남성_hippie", "남성_ivy", "남성_metrosexual",
    "남성_mods", "남성_normcore", "남성_sportivecasual", "여성_athleisure",
    "여성_bodyconscious", "여성_cityglam", "여성_classic", "여성_disco",
    "여성_ecology", "여성_feminine", "여성_genderless", "여성_grunge",
    "여성_hiphop", "여성_hippie", "여성_kitsch", "여성_lingerie",
    "여성_lounge", "여성_military", "여성_minimal", "여성_normcore",
    "여성_oriental", "여성_popart", "여성_powersuit", "여성_punk",
    "여성_space", "여성_sportivecasual"
]

# 스타일 매핑 (각 클래스 조합을 숫자로 매핑)
style_mapping = {style: idx for idx, style in enumerate(fashion_classes)}

# 데이터셋 클래스 정의
class CustomImageDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
        self.transform = transform
        self.labels = []

        # 파일명에서 성별 및 스타일 레이블 추출
        for img in self.image_paths:
            fname = os.path.basename(img)
            parts = fname.split('_')
            gender = parts[-1].split('.')[0]  # 성별 추출 ('M' 또는 'W')
            style = parts[-2]  # 스타일 추출

            gender_label = "남성" if gender == 'M' else "여성"
            label = f"{gender_label}_{style}"  # 성별 + 스타일 조합
            style_label = style_mapping.get(label, -1)  # 스타일 매핑

            if style_label != -1:
                self.labels.append(style_label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        label = self.labels[idx]
        return img, label

# 트레이닝 데이터 증강 강화
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 좌우 뒤집기
    transforms.RandomRotation(degrees=30),  # 회전 범위 증가
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 색상 조정
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 무작위 자르기 및 리사이즈
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 검증 데이터 증강 제거
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 트레이닝 데이터셋 로드
train_dataset = CustomImageDataset('/content/drive/MyDrive/dataset/processed_segmentation_cleaned', transform=train_transform)

# 검증 데이터셋 로드 (별도의 검증 이미지 폴더 사용)
val_dataset = CustomImageDataset('/content/drive/MyDrive/dataset/processed_segmentation_cleaned_for_val', transform=val_transform)

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# ResNet-18 기반 패션 스타일 및 성별 예측 모델
class FashionGenderModel(nn.Module):
    def __init__(self, num_classes=31):  # 총 31개의 클래스
        super(FashionGenderModel, self).__init__()
        self.resnet = models.resnet18(weights=None)  # Pretrained weights 사용 안 함
        self.resnet.fc = nn.Identity()  # ResNet의 기본 fully connected 레이어를 제거

        # 31개 클래스(성별 + 스타일 조합)를 위한 출력 레이어 추가
        self.fc = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(p=0.4)  # Dropout 비율 조정 (0.4)

    def forward(self, x):
        x = self.resnet(x)
        x = self.dropout(x)
        out = self.fc(x)  # 31개 클래스 예측
        return out

# 모델, 손실 함수, 옵티마이저, 스케줄러 설정
model = FashionGenderModel(num_classes=31).to(device)  # 모델을 CUDA로 이동
criterion = nn.CrossEntropyLoss()  # 하나의 CrossEntropyLoss 사용

# AdamP 옵티마이저 설정
optimizer = optim.AdamP(model.parameters(), lr=0.005, weight_decay=0.02)

# 학습률 스케줄러 설정
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)

# 모델 학습 함수
def train_model(train_loader, val_loader, model, optimizer, criterion, scheduler, epochs=300, save_path='/content/drive/MyDrive/진킴/model_final5.pth', start_epoch=0):
    best_val_accuracy = 0.0  # 최상의 검증 정확도를 저장하기 위한 변수

    # 저장된 모델이 있는 경우 불러오기
    if os.path.exists(save_path):
        print(f"Loading saved model from {save_path}...")
        model.load_state_dict(torch.load(save_path))
        print("Model loaded successfully!")

    for epoch in range(start_epoch, epochs):
        model.train()
        total_train_loss = 0
        correct_train = 0
        total_train = 0

        # 학습 단계
        for images, labels in train_loader:
            images = images.to(device)  # 데이터를 CUDA로 이동
            labels = labels.to(device)

            optimizer.zero_grad()

            # 모델 예측
            outputs = model(images)

            # 손실 계산
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            # 정확도 계산
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_accuracy = correct_train / total_train

        # 검증 단계
        model.eval()
        total_val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images = val_images.to(device)  # 검증 데이터도 CUDA로 이동
                val_labels = val_labels.to(device)

                val_outputs = model(val_images)

                val_loss = criterion(val_outputs, val_labels)
                total_val_loss += val_loss.item()

                _, val_predicted = torch.max(val_outputs, 1)
                correct_val += (val_predicted == val_labels).sum().item()
                total_val += val_labels.size(0)

        val_accuracy = correct_val / total_val

        # 학습률을 퍼센트로 표시
        lr_percent = optimizer.param_groups[0]['lr'] * 100
        scheduler.step()

        # 에포크 마다 출력
        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader):.4f}, '
              f'Train Accuracy: {train_accuracy * 100:.2f}%, '
              f'Validation Loss: {total_val_loss / len(val_loader):.4f}, '
              f'Validation Accuracy: {val_accuracy * 100:.2f}%, '
              f'Learning Rate: {lr_percent:.2f}%')
        # 최상의 검증 정확도를 가진 모델 저장
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), save_path)
            print(f'Model saved with Validation Accuracy: {val_accuracy * 100:.2f}%')

# 모델 학습 시작
train_model(train_loader, val_loader, model, optimizer, criterion, scheduler, epochs=300, save_path='/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/model_final5.pth')


Loading saved model from /content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/model_final5.pth...


<ipython-input-5-a9d2595e4d2d>:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


Model loaded successfully!
Epoch 1, Train Loss: 0.9508, Train Accuracy: 71.62%, Validation Loss: 3.3253, Validation Accuracy: 37.22%, Learning Rate: 0.50%
Model saved with Validation Accuracy: 37.22%
Epoch 2, Train Loss: 0.7871, Train Accuracy: 75.06%, Validation Loss: 2.4328, Validation Accuracy: 50.89%, Learning Rate: 0.49%
Model saved with Validation Accuracy: 50.89%
Epoch 3, Train Loss: 0.6034, Train Accuracy: 81.11%, Validation Loss: 2.3584, Validation Accuracy: 54.36%, Learning Rate: 0.45%
Model saved with Validation Accuracy: 54.36%
Epoch 4, Train Loss: 0.5124, Train Accuracy: 83.96%, Validation Loss: 2.3095, Validation Accuracy: 58.57%, Learning Rate: 0.40%
Model saved with Validation Accuracy: 58.57%
Epoch 5, Train Loss: 0.4264, Train Accuracy: 86.93%, Validation Loss: 2.5402, Validation Accuracy: 57.73%, Learning Rate: 0.33%
Epoch 6, Train Loss: 0.3088, Train Accuracy: 90.93%, Validation Loss: 2.1602, Validation Accuracy: 61.51%, Learning Rate: 0.25%
Model saved with Validati

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



## 현재 결과는 재로드된 모델 p

## BEST SCORE => 64.14%

## 1~210 에폭 실행 결과.

Epoch 1, Train Loss: 3.4663, Train Accuracy: 7.32%, Validation Loss: 3.1772, Validation Accuracy: 10.09%, Learning Rate: 0.50%
Model saved with Validation Accuracy: 10.09%
Epoch 2, Train Loss: 3.2314, Train Accuracy: 8.82%, Validation Loss: 3.1504, Validation Accuracy: 6.73%, Learning Rate: 0.49%
Epoch 3, Train Loss: 3.1916, Train Accuracy: 8.97%, Validation Loss: 3.1084, Validation Accuracy: 8.94%, Learning Rate: 0.45%
Epoch 4, Train Loss: 3.1679, Train Accuracy: 8.94%, Validation Loss: 3.1160, Validation Accuracy: 8.94%, Learning Rate: 0.40%
Epoch 5, Train Loss: 3.1495, Train Accuracy: 9.53%, Validation Loss: 3.0940, Validation Accuracy: 7.47%, Learning Rate: 0.33%
Epoch 6, Train Loss: 3.1186, Train Accuracy: 9.68%, Validation Loss: 3.0751, Validation Accuracy: 10.62%, Learning Rate: 0.25%
Model saved with Validation Accuracy: 10.62%
Epoch 7, Train Loss: 3.1092, Train Accuracy: 10.00%, Validation Loss: 3.0494, Validation Accuracy: 10.30%, Learning Rate: 0.17%
Epoch 8, Train Loss: 3.0825, Train Accuracy: 10.98%, Validation Loss: 3.0716, Validation Accuracy: 9.67%, Learning Rate: 0.10%
Epoch 9, Train Loss: 3.0612, Train Accuracy: 11.15%, Validation Loss: 3.0390, Validation Accuracy: 11.15%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 11.15%
Epoch 10, Train Loss: 3.0398, Train Accuracy: 11.62%, Validation Loss: 3.0220, Validation Accuracy: 11.99%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 11.99%
Epoch 11, Train Loss: 3.1480, Train Accuracy: 8.94%, Validation Loss: 3.1235, Validation Accuracy: 7.36%, Learning Rate: 0.50%
Epoch 12, Train Loss: 3.1314, Train Accuracy: 9.46%, Validation Loss: 3.0657, Validation Accuracy: 8.73%, Learning Rate: 0.49%
Epoch 13, Train Loss: 3.1154, Train Accuracy: 10.22%, Validation Loss: 3.1156, Validation Accuracy: 6.83%, Learning Rate: 0.45%
Epoch 14, Train Loss: 3.1076, Train Accuracy: 9.75%, Validation Loss: 3.1239, Validation Accuracy: 5.78%, Learning Rate: 0.40%
Epoch 15, Train Loss: 3.0853, Train Accuracy: 10.10%, Validation Loss: 3.0584, Validation Accuracy: 11.25%, Learning Rate: 0.33%
Epoch 16, Train Loss: 3.0768, Train Accuracy: 10.96%, Validation Loss: 3.7500, Validation Accuracy: 9.04%, Learning Rate: 0.25%
Epoch 17, Train Loss: 3.0523, Train Accuracy: 11.33%, Validation Loss: 2.9880, Validation Accuracy: 12.20%, Learning Rate: 0.17%
Model saved with Validation Accuracy: 12.20%
Epoch 18, Train Loss: 3.0221, Train Accuracy: 11.15%, Validation Loss: 2.9888, Validation Accuracy: 11.57%, Learning Rate: 0.10%
Epoch 19, Train Loss: 2.9889, Train Accuracy: 12.11%, Validation Loss: 3.0026, Validation Accuracy: 11.88%, Learning Rate: 0.05%
Epoch 20, Train Loss: 2.9598, Train Accuracy: 13.00%, Validation Loss: 2.9678, Validation Accuracy: 12.30%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 12.30%
Epoch 21, Train Loss: 3.1093, Train Accuracy: 9.83%, Validation Loss: 3.1159, Validation Accuracy: 9.15%, Learning Rate: 0.50%
Epoch 22, Train Loss: 3.1044, Train Accuracy: 10.32%, Validation Loss: 3.1442, Validation Accuracy: 10.30%, Learning Rate: 0.49%
Epoch 23, Train Loss: 3.0878, Train Accuracy: 10.20%, Validation Loss: 3.0245, Validation Accuracy: 11.04%, Learning Rate: 0.45%
Epoch 24, Train Loss: 3.0555, Train Accuracy: 11.62%, Validation Loss: 3.2219, Validation Accuracy: 9.25%, Learning Rate: 0.40%
Epoch 25, Train Loss: 3.0235, Train Accuracy: 11.35%, Validation Loss: 3.1496, Validation Accuracy: 11.99%, Learning Rate: 0.33%
Epoch 26, Train Loss: 3.0166, Train Accuracy: 11.77%, Validation Loss: 3.0155, Validation Accuracy: 10.94%, Learning Rate: 0.25%
Epoch 27, Train Loss: 2.9613, Train Accuracy: 13.96%, Validation Loss: 2.9157, Validation Accuracy: 13.88%, Learning Rate: 0.17%
Model saved with Validation Accuracy: 13.88%
Epoch 28, Train Loss: 2.9423, Train Accuracy: 13.51%, Validation Loss: 2.9462, Validation Accuracy: 13.25%, Learning Rate: 0.10%
Epoch 29, Train Loss: 2.9006, Train Accuracy: 15.18%, Validation Loss: 2.8681, Validation Accuracy: 14.72%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 14.72%
Epoch 30, Train Loss: 2.8622, Train Accuracy: 14.67%, Validation Loss: 2.8456, Validation Accuracy: 15.25%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 15.25%
Epoch 31, Train Loss: 3.0743, Train Accuracy: 10.49%, Validation Loss: 3.2270, Validation Accuracy: 7.99%, Learning Rate: 0.50%
Epoch 32, Train Loss: 3.0405, Train Accuracy: 11.23%, Validation Loss: 3.5236, Validation Accuracy: 8.41%, Learning Rate: 0.49%
Epoch 33, Train Loss: 3.0282, Train Accuracy: 11.84%, Validation Loss: 3.5930, Validation Accuracy: 6.52%, Learning Rate: 0.45%
Epoch 34, Train Loss: 3.0125, Train Accuracy: 11.89%, Validation Loss: 3.3190, Validation Accuracy: 7.26%, Learning Rate: 0.40%
Epoch 35, Train Loss: 2.9919, Train Accuracy: 11.77%, Validation Loss: 3.0180, Validation Accuracy: 11.67%, Learning Rate: 0.33%
Epoch 36, Train Loss: 2.9532, Train Accuracy: 12.63%, Validation Loss: 2.9305, Validation Accuracy: 13.99%, Learning Rate: 0.25%
Epoch 37, Train Loss: 2.9008, Train Accuracy: 13.32%, Validation Loss: 3.0025, Validation Accuracy: 13.14%, Learning Rate: 0.17%
Epoch 38, Train Loss: 2.8536, Train Accuracy: 14.86%, Validation Loss: 2.8455, Validation Accuracy: 14.62%, Learning Rate: 0.10%
Epoch 39, Train Loss: 2.7994, Train Accuracy: 15.97%, Validation Loss: 2.7802, Validation Accuracy: 17.14%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 17.14%
Epoch 40, Train Loss: 2.7609, Train Accuracy: 17.76%, Validation Loss: 2.7348, Validation Accuracy: 18.61%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 18.61%
Epoch 41, Train Loss: 2.9830, Train Accuracy: 12.11%, Validation Loss: 6.2685, Validation Accuracy: 9.15%, Learning Rate: 0.50%
Epoch 42, Train Loss: 2.9932, Train Accuracy: 12.73%, Validation Loss: 3.5409, Validation Accuracy: 8.20%, Learning Rate: 0.49%
Epoch 43, Train Loss: 2.9611, Train Accuracy: 12.78%, Validation Loss: 3.6492, Validation Accuracy: 6.41%, Learning Rate: 0.45%
Epoch 44, Train Loss: 2.9554, Train Accuracy: 11.87%, Validation Loss: 3.2587, Validation Accuracy: 8.62%, Learning Rate: 0.40%
Epoch 45, Train Loss: 2.9079, Train Accuracy: 14.69%, Validation Loss: 3.3203, Validation Accuracy: 9.36%, Learning Rate: 0.33%
Epoch 46, Train Loss: 2.8594, Train Accuracy: 14.94%, Validation Loss: 2.9498, Validation Accuracy: 14.20%, Learning Rate: 0.25%
Epoch 47, Train Loss: 2.8157, Train Accuracy: 15.80%, Validation Loss: 2.9870, Validation Accuracy: 12.20%, Learning Rate: 0.17%
Epoch 48, Train Loss: 2.7480, Train Accuracy: 17.67%, Validation Loss: 2.7833, Validation Accuracy: 16.61%, Learning Rate: 0.10%
Epoch 49, Train Loss: 2.6877, Train Accuracy: 18.82%, Validation Loss: 2.6860, Validation Accuracy: 18.30%, Learning Rate: 0.05%
Epoch 50, Train Loss: 2.6526, Train Accuracy: 19.34%, Validation Loss: 2.6475, Validation Accuracy: 18.61%, Learning Rate: 0.01%
Epoch 51, Train Loss: 2.9142, Train Accuracy: 13.73%, Validation Loss: 3.5068, Validation Accuracy: 7.15%, Learning Rate: 0.50%
Epoch 52, Train Loss: 2.9055, Train Accuracy: 13.56%, Validation Loss: 3.1434, Validation Accuracy: 9.67%, Learning Rate: 0.49%
Epoch 53, Train Loss: 2.8982, Train Accuracy: 14.45%, Validation Loss: 4.1728, Validation Accuracy: 5.99%, Learning Rate: 0.45%
Epoch 54, Train Loss: 2.8468, Train Accuracy: 15.18%, Validation Loss: 3.0330, Validation Accuracy: 11.15%, Learning Rate: 0.40%
Epoch 55, Train Loss: 2.8292, Train Accuracy: 15.23%, Validation Loss: 4.2324, Validation Accuracy: 6.10%, Learning Rate: 0.33%
Epoch 56, Train Loss: 2.7948, Train Accuracy: 16.49%, Validation Loss: 2.9269, Validation Accuracy: 14.93%, Learning Rate: 0.25%
Epoch 57, Train Loss: 2.7246, Train Accuracy: 17.49%, Validation Loss: 2.9857, Validation Accuracy: 12.72%, Learning Rate: 0.17%
Epoch 58, Train Loss: 2.6601, Train Accuracy: 19.75%, Validation Loss: 2.8334, Validation Accuracy: 14.83%, Learning Rate: 0.10%
Epoch 59, Train Loss: 2.5973, Train Accuracy: 21.13%, Validation Loss: 2.6645, Validation Accuracy: 19.35%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 19.35%
Epoch 60, Train Loss: 2.5698, Train Accuracy: 20.66%, Validation Loss: 2.5779, Validation Accuracy: 21.87%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 21.87%
Epoch 61, Train Loss: 2.8366, Train Accuracy: 15.50%, Validation Loss: 6.0041, Validation Accuracy: 9.04%, Learning Rate: 0.50%
Epoch 62, Train Loss: 2.8649, Train Accuracy: 15.09%, Validation Loss: 2.9742, Validation Accuracy: 11.25%, Learning Rate: 0.49%
Epoch 63, Train Loss: 2.8259, Train Accuracy: 15.77%, Validation Loss: 3.2828, Validation Accuracy: 13.35%, Learning Rate: 0.45%
Epoch 64, Train Loss: 2.8225, Train Accuracy: 15.45%, Validation Loss: 2.7892, Validation Accuracy: 15.88%, Learning Rate: 0.40%
Epoch 65, Train Loss: 2.7525, Train Accuracy: 16.58%, Validation Loss: 3.1659, Validation Accuracy: 9.67%, Learning Rate: 0.33%
Epoch 66, Train Loss: 2.7179, Train Accuracy: 18.08%, Validation Loss: 2.6430, Validation Accuracy: 19.45%, Learning Rate: 0.25%
Epoch 67, Train Loss: 2.6360, Train Accuracy: 19.66%, Validation Loss: 4.5873, Validation Accuracy: 4.94%, Learning Rate: 0.17%
Epoch 68, Train Loss: 2.5686, Train Accuracy: 21.50%, Validation Loss: 4.2480, Validation Accuracy: 6.41%, Learning Rate: 0.10%
Epoch 69, Train Loss: 2.5190, Train Accuracy: 23.54%, Validation Loss: 2.5330, Validation Accuracy: 23.55%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 23.55%
Epoch 70, Train Loss: 2.4754, Train Accuracy: 23.51%, Validation Loss: 2.5487, Validation Accuracy: 21.77%, Learning Rate: 0.01%
Epoch 71, Train Loss: 2.7655, Train Accuracy: 16.54%, Validation Loss: 4.1402, Validation Accuracy: 6.10%, Learning Rate: 0.50%
Epoch 72, Train Loss: 2.7941, Train Accuracy: 16.90%, Validation Loss: 3.6412, Validation Accuracy: 6.10%, Learning Rate: 0.49%
Epoch 73, Train Loss: 2.7850, Train Accuracy: 16.44%, Validation Loss: 4.2531, Validation Accuracy: 6.10%, Learning Rate: 0.45%
Epoch 74, Train Loss: 2.7380, Train Accuracy: 17.64%, Validation Loss: 3.5508, Validation Accuracy: 7.68%, Learning Rate: 0.40%
Epoch 75, Train Loss: 2.7082, Train Accuracy: 18.55%, Validation Loss: 4.8181, Validation Accuracy: 5.99%, Learning Rate: 0.33%
Epoch 76, Train Loss: 2.6452, Train Accuracy: 19.16%, Validation Loss: 2.6157, Validation Accuracy: 21.24%, Learning Rate: 0.25%
Epoch 77, Train Loss: 2.5840, Train Accuracy: 20.96%, Validation Loss: 3.3389, Validation Accuracy: 14.20%, Learning Rate: 0.17%
Epoch 78, Train Loss: 2.5115, Train Accuracy: 22.53%, Validation Loss: 3.7727, Validation Accuracy: 8.10%, Learning Rate: 0.10%
Epoch 79, Train Loss: 2.4376, Train Accuracy: 24.82%, Validation Loss: 2.4746, Validation Accuracy: 24.29%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 24.29%
Epoch 80, Train Loss: 2.3864, Train Accuracy: 26.34%, Validation Loss: 2.4348, Validation Accuracy: 24.82%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 24.82%
Epoch 81, Train Loss: 2.8083, Train Accuracy: 16.24%, Validation Loss: 3.0772, Validation Accuracy: 12.51%, Learning Rate: 0.50%
Epoch 82, Train Loss: 2.8211, Train Accuracy: 16.14%, Validation Loss: 3.0748, Validation Accuracy: 11.25%, Learning Rate: 0.49%
Epoch 83, Train Loss: 2.7610, Train Accuracy: 17.15%, Validation Loss: 4.0104, Validation Accuracy: 6.94%, Learning Rate: 0.45%
Epoch 84, Train Loss: 2.7262, Train Accuracy: 17.79%, Validation Loss: 4.0074, Validation Accuracy: 4.31%, Learning Rate: 0.40%
Epoch 85, Train Loss: 2.6724, Train Accuracy: 18.01%, Validation Loss: 4.1713, Validation Accuracy: 6.10%, Learning Rate: 0.33%
Epoch 86, Train Loss: 2.6113, Train Accuracy: 20.57%, Validation Loss: 2.8933, Validation Accuracy: 15.98%, Learning Rate: 0.25%
Epoch 87, Train Loss: 2.5563, Train Accuracy: 22.04%, Validation Loss: 2.5416, Validation Accuracy: 21.77%, Learning Rate: 0.17%
Epoch 88, Train Loss: 2.4642, Train Accuracy: 23.56%, Validation Loss: 3.5174, Validation Accuracy: 12.83%, Learning Rate: 0.10%
Epoch 89, Train Loss: 2.3829, Train Accuracy: 26.09%, Validation Loss: 2.4291, Validation Accuracy: 26.71%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 26.71%
Epoch 90, Train Loss: 2.3486, Train Accuracy: 26.17%, Validation Loss: 2.4049, Validation Accuracy: 26.08%, Learning Rate: 0.01%
Epoch 91, Train Loss: 2.7065, Train Accuracy: 18.53%, Validation Loss: 6.0996, Validation Accuracy: 2.84%, Learning Rate: 0.50%
Epoch 92, Train Loss: 2.7312, Train Accuracy: 17.13%, Validation Loss: 3.4842, Validation Accuracy: 6.41%, Learning Rate: 0.49%
Epoch 93, Train Loss: 2.7011, Train Accuracy: 18.06%, Validation Loss: 5.7472, Validation Accuracy: 5.99%, Learning Rate: 0.45%
Epoch 94, Train Loss: 2.6748, Train Accuracy: 19.53%, Validation Loss: 4.3593, Validation Accuracy: 5.89%, Learning Rate: 0.40%
Epoch 95, Train Loss: 2.6048, Train Accuracy: 21.03%, Validation Loss: 5.4017, Validation Accuracy: 4.63%, Learning Rate: 0.33%
Epoch 96, Train Loss: 2.5269, Train Accuracy: 22.21%, Validation Loss: 2.6972, Validation Accuracy: 18.61%, Learning Rate: 0.25%
Epoch 97, Train Loss: 2.4487, Train Accuracy: 24.37%, Validation Loss: 2.5884, Validation Accuracy: 23.24%, Learning Rate: 0.17%
Epoch 98, Train Loss: 2.3770, Train Accuracy: 26.39%, Validation Loss: 2.4516, Validation Accuracy: 25.03%, Learning Rate: 0.10%
Epoch 99, Train Loss: 2.2738, Train Accuracy: 28.99%, Validation Loss: 3.3474, Validation Accuracy: 12.93%, Learning Rate: 0.05%
Epoch 100, Train Loss: 2.2280, Train Accuracy: 30.47%, Validation Loss: 2.3387, Validation Accuracy: 27.34%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 27.34%
Epoch 101, Train Loss: 2.6553, Train Accuracy: 20.61%, Validation Loss: 3.8495, Validation Accuracy: 7.26%, Learning Rate: 0.50%
Epoch 102, Train Loss: 2.6996, Train Accuracy: 18.97%, Validation Loss: 4.6220, Validation Accuracy: 5.99%, Learning Rate: 0.49%
Epoch 103, Train Loss: 2.6864, Train Accuracy: 19.41%, Validation Loss: 2.7376, Validation Accuracy: 17.25%, Learning Rate: 0.45%
Epoch 104, Train Loss: 2.6225, Train Accuracy: 19.61%, Validation Loss: 3.0538, Validation Accuracy: 14.83%, Learning Rate: 0.40%
Epoch 105, Train Loss: 2.5826, Train Accuracy: 21.38%, Validation Loss: 2.8658, Validation Accuracy: 15.04%, Learning Rate: 0.33%
Epoch 106, Train Loss: 2.4879, Train Accuracy: 22.60%, Validation Loss: 2.5646, Validation Accuracy: 21.03%, Learning Rate: 0.25%
Epoch 107, Train Loss: 2.4102, Train Accuracy: 26.14%, Validation Loss: 2.4501, Validation Accuracy: 24.19%, Learning Rate: 0.17%
Epoch 108, Train Loss: 2.2803, Train Accuracy: 29.98%, Validation Loss: 2.3659, Validation Accuracy: 25.87%, Learning Rate: 0.10%
Epoch 109, Train Loss: 2.1895, Train Accuracy: 30.34%, Validation Loss: 2.2697, Validation Accuracy: 29.23%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 29.23%
Epoch 110, Train Loss: 2.1133, Train Accuracy: 33.10%, Validation Loss: 2.2624, Validation Accuracy: 28.39%, Learning Rate: 0.01%
Epoch 111, Train Loss: 2.6320, Train Accuracy: 20.39%, Validation Loss: 5.4777, Validation Accuracy: 5.99%, Learning Rate: 0.50%
Epoch 112, Train Loss: 2.6670, Train Accuracy: 19.48%, Validation Loss: 5.0101, Validation Accuracy: 5.78%, Learning Rate: 0.49%
Epoch 113, Train Loss: 2.5851, Train Accuracy: 21.52%, Validation Loss: 2.7461, Validation Accuracy: 18.40%, Learning Rate: 0.45%
Epoch 114, Train Loss: 2.5515, Train Accuracy: 21.65%, Validation Loss: 2.7049, Validation Accuracy: 20.40%, Learning Rate: 0.40%
Epoch 115, Train Loss: 2.4929, Train Accuracy: 23.78%, Validation Loss: 2.6291, Validation Accuracy: 20.29%, Learning Rate: 0.33%
Epoch 116, Train Loss: 2.3647, Train Accuracy: 26.14%, Validation Loss: 4.6654, Validation Accuracy: 7.57%, Learning Rate: 0.25%
Epoch 117, Train Loss: 2.2900, Train Accuracy: 28.87%, Validation Loss: 2.5517, Validation Accuracy: 24.19%, Learning Rate: 0.17%
Epoch 118, Train Loss: 2.1673, Train Accuracy: 31.52%, Validation Loss: 2.2941, Validation Accuracy: 30.28%, Learning Rate: 0.10%
Model saved with Validation Accuracy: 30.28%
Epoch 119, Train Loss: 2.0369, Train Accuracy: 36.68%, Validation Loss: 2.1890, Validation Accuracy: 34.17%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 34.17%
Epoch 120, Train Loss: 1.9426, Train Accuracy: 38.94%, Validation Loss: 2.1795, Validation Accuracy: 33.54%, Learning Rate: 0.01%
Epoch 121, Train Loss: 2.5101, Train Accuracy: 24.59%, Validation Loss: 2.8003, Validation Accuracy: 18.93%, Learning Rate: 0.50%
Epoch 122, Train Loss: 2.5860, Train Accuracy: 20.91%, Validation Loss: 3.0289, Validation Accuracy: 14.83%, Learning Rate: 0.49%
Epoch 123, Train Loss: 2.5340, Train Accuracy: 22.33%, Validation Loss: 3.0094, Validation Accuracy: 15.46%, Learning Rate: 0.45%
Epoch 124, Train Loss: 2.4739, Train Accuracy: 23.56%, Validation Loss: 4.1949, Validation Accuracy: 8.94%, Learning Rate: 0.40%
Epoch 125, Train Loss: 2.3990, Train Accuracy: 25.16%, Validation Loss: 6.2985, Validation Accuracy: 6.31%, Learning Rate: 0.33%
Epoch 126, Train Loss: 2.2927, Train Accuracy: 28.92%, Validation Loss: 4.4978, Validation Accuracy: 7.15%, Learning Rate: 0.25%
Epoch 127, Train Loss: 2.1596, Train Accuracy: 32.41%, Validation Loss: 2.2800, Validation Accuracy: 29.97%, Learning Rate: 0.17%
Epoch 128, Train Loss: 2.0058, Train Accuracy: 36.93%, Validation Loss: 6.0727, Validation Accuracy: 8.52%, Learning Rate: 0.10%
Epoch 129, Train Loss: 1.8851, Train Accuracy: 40.69%, Validation Loss: 2.1018, Validation Accuracy: 36.91%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 36.91%
Epoch 130, Train Loss: 1.7812, Train Accuracy: 44.03%, Validation Loss: 2.0509, Validation Accuracy: 38.49%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 38.49%
Epoch 131, Train Loss: 2.5021, Train Accuracy: 24.57%, Validation Loss: 6.0328, Validation Accuracy: 5.57%, Learning Rate: 0.50%
Epoch 132, Train Loss: 2.5585, Train Accuracy: 22.19%, Validation Loss: 3.2446, Validation Accuracy: 12.30%, Learning Rate: 0.49%
Epoch 133, Train Loss: 2.4793, Train Accuracy: 23.76%, Validation Loss: 4.1814, Validation Accuracy: 6.20%, Learning Rate: 0.45%
Epoch 134, Train Loss: 2.4193, Train Accuracy: 25.11%, Validation Loss: 6.7185, Validation Accuracy: 5.78%, Learning Rate: 0.40%
Epoch 135, Train Loss: 2.3384, Train Accuracy: 27.67%, Validation Loss: 4.6271, Validation Accuracy: 7.99%, Learning Rate: 0.33%
Epoch 136, Train Loss: 2.1758, Train Accuracy: 32.16%, Validation Loss: 2.3515, Validation Accuracy: 27.87%, Learning Rate: 0.25%
Epoch 137, Train Loss: 2.0334, Train Accuracy: 36.46%, Validation Loss: 5.5695, Validation Accuracy: 6.31%, Learning Rate: 0.17%
Epoch 138, Train Loss: 1.8764, Train Accuracy: 40.79%, Validation Loss: 2.1311, Validation Accuracy: 35.75%, Learning Rate: 0.10%
Epoch 139, Train Loss: 1.7062, Train Accuracy: 46.04%, Validation Loss: 2.2946, Validation Accuracy: 32.49%, Learning Rate: 0.05%
Epoch 140, Train Loss: 1.5926, Train Accuracy: 50.10%, Validation Loss: 1.9789, Validation Accuracy: 42.90%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 42.90%
Epoch 141, Train Loss: 2.4057, Train Accuracy: 26.46%, Validation Loss: 5.7100, Validation Accuracy: 6.10%, Learning Rate: 0.50%
Epoch 142, Train Loss: 2.5313, Train Accuracy: 22.85%, Validation Loss: 2.6607, Validation Accuracy: 21.45%, Learning Rate: 0.49%
Epoch 143, Train Loss: 2.4165, Train Accuracy: 25.48%, Validation Loss: 4.2636, Validation Accuracy: 5.05%, Learning Rate: 0.45%
Epoch 144, Train Loss: 2.3326, Train Accuracy: 28.26%, Validation Loss: 2.7531, Validation Accuracy: 20.61%, Learning Rate: 0.40%
Epoch 145, Train Loss: 2.2376, Train Accuracy: 30.15%, Validation Loss: 3.7592, Validation Accuracy: 14.72%, Learning Rate: 0.33%
Epoch 146, Train Loss: 2.0662, Train Accuracy: 34.94%, Validation Loss: 4.7301, Validation Accuracy: 6.41%, Learning Rate: 0.25%
Epoch 147, Train Loss: 1.8691, Train Accuracy: 41.03%, Validation Loss: 2.2836, Validation Accuracy: 34.38%, Learning Rate: 0.17%
Epoch 148, Train Loss: 1.6958, Train Accuracy: 47.30%, Validation Loss: 1.9852, Validation Accuracy: 42.80%, Learning Rate: 0.10%
Epoch 149, Train Loss: 1.5242, Train Accuracy: 51.70%, Validation Loss: 1.9063, Validation Accuracy: 47.95%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 47.95%
Epoch 150, Train Loss: 1.4039, Train Accuracy: 56.81%, Validation Loss: 1.8582, Validation Accuracy: 49.95%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 49.95%
Epoch 151, Train Loss: 2.2632, Train Accuracy: 31.55%, Validation Loss: 2.6859, Validation Accuracy: 17.35%, Learning Rate: 0.50%
Epoch 152, Train Loss: 2.4618, Train Accuracy: 24.23%, Validation Loss: 2.7248, Validation Accuracy: 20.72%, Learning Rate: 0.49%
Epoch 153, Train Loss: 2.3761, Train Accuracy: 26.46%, Validation Loss: 4.6959, Validation Accuracy: 7.47%, Learning Rate: 0.45%
Epoch 154, Train Loss: 2.2112, Train Accuracy: 29.75%, Validation Loss: 4.5713, Validation Accuracy: 8.52%, Learning Rate: 0.40%
Epoch 155, Train Loss: 2.1300, Train Accuracy: 34.20%, Validation Loss: 2.4556, Validation Accuracy: 28.08%, Learning Rate: 0.33%
Epoch 156, Train Loss: 1.9436, Train Accuracy: 38.65%, Validation Loss: 2.3550, Validation Accuracy: 29.76%, Learning Rate: 0.25%
Epoch 157, Train Loss: 1.7492, Train Accuracy: 45.14%, Validation Loss: 2.0155, Validation Accuracy: 44.16%, Learning Rate: 0.17%
Epoch 158, Train Loss: 1.5071, Train Accuracy: 53.10%, Validation Loss: 1.8961, Validation Accuracy: 47.42%, Learning Rate: 0.10%
Epoch 159, Train Loss: 1.2930, Train Accuracy: 60.47%, Validation Loss: 1.8094, Validation Accuracy: 51.95%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 51.95%
Epoch 160, Train Loss: 1.1850, Train Accuracy: 64.59%, Validation Loss: 1.7655, Validation Accuracy: 52.37%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 52.37%
Epoch 161, Train Loss: 2.2295, Train Accuracy: 31.92%, Validation Loss: 4.6216, Validation Accuracy: 3.68%, Learning Rate: 0.50%
Epoch 162, Train Loss: 2.4273, Train Accuracy: 25.58%, Validation Loss: 6.0453, Validation Accuracy: 7.05%, Learning Rate: 0.49%
Epoch 163, Train Loss: 2.2920, Train Accuracy: 29.66%, Validation Loss: 2.8013, Validation Accuracy: 21.66%, Learning Rate: 0.45%
Epoch 164, Train Loss: 2.1225, Train Accuracy: 33.78%, Validation Loss: 2.3587, Validation Accuracy: 29.23%, Learning Rate: 0.40%
Epoch 165, Train Loss: 1.9606, Train Accuracy: 38.01%, Validation Loss: 2.3932, Validation Accuracy: 28.92%, Learning Rate: 0.33%
Epoch 166, Train Loss: 1.7739, Train Accuracy: 43.49%, Validation Loss: 5.1708, Validation Accuracy: 7.78%, Learning Rate: 0.25%
Epoch 167, Train Loss: 1.5504, Train Accuracy: 51.50%, Validation Loss: 1.9519, Validation Accuracy: 47.95%, Learning Rate: 0.17%
Epoch 168, Train Loss: 1.2936, Train Accuracy: 59.63%, Validation Loss: 1.9195, Validation Accuracy: 49.95%, Learning Rate: 0.10%
Epoch 169, Train Loss: 1.1326, Train Accuracy: 65.11%, Validation Loss: 1.7676, Validation Accuracy: 54.36%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 54.36%
Epoch 170, Train Loss: 1.0155, Train Accuracy: 69.80%, Validation Loss: 1.7554, Validation Accuracy: 55.42%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 55.42%
Epoch 171, Train Loss: 1.9807, Train Accuracy: 40.17%, Validation Loss: 9.0600, Validation Accuracy: 6.10%, Learning Rate: 0.50%
Epoch 172, Train Loss: 2.3354, Train Accuracy: 27.49%, Validation Loss: 4.8622, Validation Accuracy: 6.31%, Learning Rate: 0.49%
Epoch 173, Train Loss: 2.1517, Train Accuracy: 32.95%, Validation Loss: 2.6531, Validation Accuracy: 25.34%, Learning Rate: 0.45%
Epoch 174, Train Loss: 1.9933, Train Accuracy: 37.37%, Validation Loss: 9.3888, Validation Accuracy: 5.99%, Learning Rate: 0.40%
Epoch 175, Train Loss: 1.8467, Train Accuracy: 41.20%, Validation Loss: 2.3365, Validation Accuracy: 34.17%, Learning Rate: 0.33%
Epoch 176, Train Loss: 1.6240, Train Accuracy: 48.57%, Validation Loss: 2.1376, Validation Accuracy: 40.48%, Learning Rate: 0.25%
Epoch 177, Train Loss: 1.3714, Train Accuracy: 56.49%, Validation Loss: 6.4622, Validation Accuracy: 6.94%, Learning Rate: 0.17%
Epoch 178, Train Loss: 1.1183, Train Accuracy: 65.26%, Validation Loss: 1.8601, Validation Accuracy: 53.42%, Learning Rate: 0.10%
Epoch 179, Train Loss: 0.9373, Train Accuracy: 71.70%, Validation Loss: 1.7462, Validation Accuracy: 57.31%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 57.31%
Epoch 180, Train Loss: 0.8350, Train Accuracy: 76.76%, Validation Loss: 1.7228, Validation Accuracy: 59.10%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 59.10%
Epoch 181, Train Loss: 1.8454, Train Accuracy: 44.08%, Validation Loss: 4.2979, Validation Accuracy: 11.78%, Learning Rate: 0.50%
Epoch 182, Train Loss: 2.1792, Train Accuracy: 32.78%, Validation Loss: 2.5853, Validation Accuracy: 25.76%, Learning Rate: 0.49%
Epoch 183, Train Loss: 2.0310, Train Accuracy: 36.46%, Validation Loss: 2.7644, Validation Accuracy: 24.40%, Learning Rate: 0.45%
Epoch 184, Train Loss: 1.9279, Train Accuracy: 40.49%, Validation Loss: 5.9566, Validation Accuracy: 6.20%, Learning Rate: 0.40%
Epoch 185, Train Loss: 1.7559, Train Accuracy: 44.79%, Validation Loss: 4.1403, Validation Accuracy: 9.25%, Learning Rate: 0.33%
Epoch 186, Train Loss: 1.4338, Train Accuracy: 55.11%, Validation Loss: 3.8000, Validation Accuracy: 13.56%, Learning Rate: 0.25%
Epoch 187, Train Loss: 1.1745, Train Accuracy: 63.64%, Validation Loss: 1.8219, Validation Accuracy: 55.21%, Learning Rate: 0.17%
Epoch 188, Train Loss: 0.9597, Train Accuracy: 70.84%, Validation Loss: 3.4327, Validation Accuracy: 21.45%, Learning Rate: 0.10%
Epoch 189, Train Loss: 0.7946, Train Accuracy: 76.73%, Validation Loss: 1.7730, Validation Accuracy: 60.15%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 60.15%
Epoch 190, Train Loss: 0.6947, Train Accuracy: 80.42%, Validation Loss: 1.7082, Validation Accuracy: 60.67%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 60.67%
Epoch 191, Train Loss: 1.5893, Train Accuracy: 51.43%, Validation Loss: 5.9744, Validation Accuracy: 7.26%, Learning Rate: 0.50%
Epoch 192, Train Loss: 2.1404, Train Accuracy: 34.00%, Validation Loss: 5.6132, Validation Accuracy: 7.05%, Learning Rate: 0.49%
Epoch 193, Train Loss: 1.9117, Train Accuracy: 39.36%, Validation Loss: 2.8331, Validation Accuracy: 29.13%, Learning Rate: 0.45%
Epoch 194, Train Loss: 1.7123, Train Accuracy: 45.55%, Validation Loss: 2.2967, Validation Accuracy: 37.85%, Learning Rate: 0.40%
Epoch 195, Train Loss: 1.5236, Train Accuracy: 51.50%, Validation Loss: 2.1972, Validation Accuracy: 44.06%, Learning Rate: 0.33%
Epoch 196, Train Loss: 1.2650, Train Accuracy: 60.61%, Validation Loss: 5.1810, Validation Accuracy: 10.20%, Learning Rate: 0.25%
Epoch 197, Train Loss: 0.9943, Train Accuracy: 69.21%, Validation Loss: 1.7878, Validation Accuracy: 57.10%, Learning Rate: 0.17%
Epoch 198, Train Loss: 0.7701, Train Accuracy: 76.36%, Validation Loss: 1.7506, Validation Accuracy: 60.25%, Learning Rate: 0.10%
Epoch 199, Train Loss: 0.6096, Train Accuracy: 82.85%, Validation Loss: 1.7249, Validation Accuracy: 61.72%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 61.72%
Epoch 200, Train Loss: 0.5517, Train Accuracy: 84.91%, Validation Loss: 1.7373, Validation Accuracy: 61.93%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 61.93%
Epoch 201, Train Loss: 1.4805, Train Accuracy: 55.18%, Validation Loss: 5.1142, Validation Accuracy: 11.15%, Learning Rate: 0.50%
Epoch 202, Train Loss: 2.0850, Train Accuracy: 35.31%, Validation Loss: 4.1473, Validation Accuracy: 7.89%, Learning Rate: 0.49%
Epoch 203, Train Loss: 1.7918, Train Accuracy: 44.28%, Validation Loss: 2.3408, Validation Accuracy: 34.28%, Learning Rate: 0.45%
Epoch 204, Train Loss: 1.5429, Train Accuracy: 51.74%, Validation Loss: 2.1132, Validation Accuracy: 44.48%, Learning Rate: 0.40%
Epoch 205, Train Loss: 1.3515, Train Accuracy: 57.27%, Validation Loss: 2.1628, Validation Accuracy: 46.06%, Learning Rate: 0.33%
Epoch 206, Train Loss: 1.1263, Train Accuracy: 64.72%, Validation Loss: 4.4192, Validation Accuracy: 15.04%, Learning Rate: 0.25%
Epoch 207, Train Loss: 0.8746, Train Accuracy: 73.91%, Validation Loss: 1.9391, Validation Accuracy: 57.31%, Learning Rate: 0.17%
Epoch 208, Train Loss: 0.6691, Train Accuracy: 79.66%, Validation Loss: 3.5393, Validation Accuracy: 24.61%, Learning Rate: 0.10%
Epoch 209, Train Loss: 0.5226, Train Accuracy: 85.31%, Validation Loss: 1.7691, Validation Accuracy: 61.93%, Learning Rate: 0.05%
Epoch 210, Train Loss: 0.4556, Train Accuracy: 87.76%, Validation Loss: 1.7964, Validation Accuracy: 61.83%, Learning Rate: 0.01%
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<ipython-input-9-fa98badd98ed> in <cell line: 194>()
    192
    193 # 모델 학습 시작
--> 194 train_model(train_loader, val_loader, model, optimizer, criterion, scheduler, epochs=300, save_path='/content/drive/MyDrive/진킴/model_final5.pth')

10 frames
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py in get_params(img, scale, ratio)
    943
    944             if 0 < w <= width and 0 < h <= height:
--> 945                 i = torch.randint(0, height - h + 1, size=(1,)).item()
    946                 j = torch.randint(0, width - w + 1, size=(1,)).item()
    947                 return i, j, h, w

KeyboardInterrupt:


## 210~400 에폭 실행 결과 => 210에서 학습 끊었다가 저장된 모델 로드해서 다시 학습 시작

Loading saved model from /content/drive/MyDrive/진킴/model_final5.pth...
<ipython-input-7-fa98badd98ed>:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))
Model loaded successfully!
Epoch 1, Train Loss: 1.5580, Train Accuracy: 51.38%, Validation Loss: 10.3372, Validation Accuracy: 6.73%, Learning Rate: 0.50%
Model saved with Validation Accuracy: 6.73%
Epoch 2, Train Loss: 1.4383, Train Accuracy: 54.35%, Validation Loss: 2.1517, Validation Accuracy: 42.80%, Learning Rate: 0.49%
Model saved with Validation Accuracy: 42.80%
Epoch 3, Train Loss: 1.2633, Train Accuracy: 59.75%, Validation Loss: 2.7484, Validation Accuracy: 35.65%, Learning Rate: 0.45%
Epoch 4, Train Loss: 1.1385, Train Accuracy: 63.66%, Validation Loss: 7.1169, Validation Accuracy: 7.68%, Learning Rate: 0.40%
Epoch 5, Train Loss: 0.9818, Train Accuracy: 69.56%, Validation Loss: 2.1541, Validation Accuracy: 51.63%, Learning Rate: 0.33%
Model saved with Validation Accuracy: 51.63%
Epoch 6, Train Loss: 0.7652, Train Accuracy: 76.76%, Validation Loss: 2.0186, Validation Accuracy: 60.15%, Learning Rate: 0.25%
Model saved with Validation Accuracy: 60.15%
Epoch 7, Train Loss: 0.6069, Train Accuracy: 81.79%, Validation Loss: 2.0728, Validation Accuracy: 56.68%, Learning Rate: 0.17%
Epoch 8, Train Loss: 0.4628, Train Accuracy: 86.36%, Validation Loss: 1.9738, Validation Accuracy: 60.78%, Learning Rate: 0.10%
Model saved with Validation Accuracy: 60.78%
Epoch 9, Train Loss: 0.3836, Train Accuracy: 89.75%, Validation Loss: 1.9083, Validation Accuracy: 61.93%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 61.93%
Epoch 10, Train Loss: 0.3371, Train Accuracy: 90.74%, Validation Loss: 1.9024, Validation Accuracy: 62.04%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 62.04%
Epoch 11, Train Loss: 0.9558, Train Accuracy: 70.79%, Validation Loss: 4.0211, Validation Accuracy: 23.24%, Learning Rate: 0.50%
Epoch 12, Train Loss: 1.7764, Train Accuracy: 45.16%, Validation Loss: 2.7372, Validation Accuracy: 29.55%, Learning Rate: 0.49%
Epoch 13, Train Loss: 1.4756, Train Accuracy: 53.61%, Validation Loss: 2.4310, Validation Accuracy: 40.69%, Learning Rate: 0.45%
Epoch 14, Train Loss: 1.2130, Train Accuracy: 62.83%, Validation Loss: 2.0787, Validation Accuracy: 49.84%, Learning Rate: 0.40%
Epoch 15, Train Loss: 1.0507, Train Accuracy: 68.55%, Validation Loss: 4.7683, Validation Accuracy: 14.30%, Learning Rate: 0.33%
Epoch 16, Train Loss: 0.7794, Train Accuracy: 75.70%, Validation Loss: 3.1659, Validation Accuracy: 31.55%, Learning Rate: 0.25%
Epoch 17, Train Loss: 0.5656, Train Accuracy: 82.46%, Validation Loss: 1.9884, Validation Accuracy: 60.04%, Learning Rate: 0.17%
Epoch 18, Train Loss: 0.4115, Train Accuracy: 87.86%, Validation Loss: 1.9507, Validation Accuracy: 61.62%, Learning Rate: 0.10%
Epoch 19, Train Loss: 0.3197, Train Accuracy: 91.18%, Validation Loss: 1.9563, Validation Accuracy: 61.83%, Learning Rate: 0.05%
Epoch 20, Train Loss: 0.3037, Train Accuracy: 91.70%, Validation Loss: 1.9436, Validation Accuracy: 62.25%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 62.25%
Epoch 21, Train Loss: 0.9235, Train Accuracy: 72.21%, Validation Loss: 2.9740, Validation Accuracy: 25.97%, Learning Rate: 0.50%
Epoch 22, Train Loss: 1.6220, Train Accuracy: 49.29%, Validation Loss: 6.6767, Validation Accuracy: 9.57%, Learning Rate: 0.49%
Epoch 23, Train Loss: 1.3890, Train Accuracy: 57.30%, Validation Loss: 4.4381, Validation Accuracy: 19.87%, Learning Rate: 0.45%
Epoch 24, Train Loss: 1.2128, Train Accuracy: 61.35%, Validation Loss: 2.5213, Validation Accuracy: 39.64%, Learning Rate: 0.40%
Epoch 25, Train Loss: 1.0326, Train Accuracy: 68.18%, Validation Loss: 3.0384, Validation Accuracy: 36.38%, Learning Rate: 0.33%
Epoch 26, Train Loss: 0.8038, Train Accuracy: 74.62%, Validation Loss: 1.9214, Validation Accuracy: 60.88%, Learning Rate: 0.25%
Epoch 27, Train Loss: 0.5492, Train Accuracy: 83.73%, Validation Loss: 1.9707, Validation Accuracy: 60.04%, Learning Rate: 0.17%
Epoch 28, Train Loss: 0.3932, Train Accuracy: 88.35%, Validation Loss: 1.9215, Validation Accuracy: 62.15%, Learning Rate: 0.10%
Epoch 29, Train Loss: 0.3060, Train Accuracy: 91.82%, Validation Loss: 1.9733, Validation Accuracy: 62.99%, Learning Rate: 0.05%
Model saved with Validation Accuracy: 62.99%
Epoch 30, Train Loss: 0.2556, Train Accuracy: 93.12%, Validation Loss: 1.9378, Validation Accuracy: 63.51%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 63.51%
Epoch 31, Train Loss: 0.7539, Train Accuracy: 77.17%, Validation Loss: 2.8117, Validation Accuracy: 41.11%, Learning Rate: 0.50%
Epoch 32, Train Loss: 1.5633, Train Accuracy: 51.77%, Validation Loss: 4.3871, Validation Accuracy: 9.15%, Learning Rate: 0.49%
Epoch 33, Train Loss: 1.3930, Train Accuracy: 55.70%, Validation Loss: 3.2248, Validation Accuracy: 28.81%, Learning Rate: 0.45%
Epoch 34, Train Loss: 1.1354, Train Accuracy: 64.20%, Validation Loss: 2.4129, Validation Accuracy: 46.69%, Learning Rate: 0.40%
Epoch 35, Train Loss: 0.9292, Train Accuracy: 71.03%, Validation Loss: 2.0759, Validation Accuracy: 56.05%, Learning Rate: 0.33%
Epoch 36, Train Loss: 0.6493, Train Accuracy: 79.75%, Validation Loss: 1.9451, Validation Accuracy: 60.15%, Learning Rate: 0.25%
Epoch 37, Train Loss: 0.4431, Train Accuracy: 87.17%, Validation Loss: 2.0262, Validation Accuracy: 58.99%, Learning Rate: 0.17%
Epoch 38, Train Loss: 0.3571, Train Accuracy: 89.71%, Validation Loss: 1.9019, Validation Accuracy: 63.72%, Learning Rate: 0.10%
Model saved with Validation Accuracy: 63.72%
Epoch 39, Train Loss: 0.2641, Train Accuracy: 92.43%, Validation Loss: 1.9525, Validation Accuracy: 62.78%, Learning Rate: 0.05%
Epoch 40, Train Loss: 0.2289, Train Accuracy: 93.98%, Validation Loss: 1.9315, Validation Accuracy: 63.20%, Learning Rate: 0.01%
Epoch 41, Train Loss: 0.7508, Train Accuracy: 77.03%, Validation Loss: 2.7431, Validation Accuracy: 45.43%, Learning Rate: 0.50%
Epoch 42, Train Loss: 1.5456, Train Accuracy: 53.14%, Validation Loss: 2.9263, Validation Accuracy: 25.97%, Learning Rate: 0.49%
Epoch 43, Train Loss: 1.4056, Train Accuracy: 55.23%, Validation Loss: 3.2257, Validation Accuracy: 30.18%, Learning Rate: 0.45%
Epoch 44, Train Loss: 1.0991, Train Accuracy: 65.36%, Validation Loss: 2.1738, Validation Accuracy: 51.42%, Learning Rate: 0.40%
Epoch 45, Train Loss: 0.8221, Train Accuracy: 73.96%, Validation Loss: 7.4588, Validation Accuracy: 8.52%, Learning Rate: 0.33%
Epoch 46, Train Loss: 0.6119, Train Accuracy: 82.14%, Validation Loss: 1.9653, Validation Accuracy: 62.36%, Learning Rate: 0.25%
Epoch 47, Train Loss: 0.4240, Train Accuracy: 87.52%, Validation Loss: 2.0236, Validation Accuracy: 63.20%, Learning Rate: 0.17%
Epoch 48, Train Loss: 0.3182, Train Accuracy: 90.54%, Validation Loss: 2.0335, Validation Accuracy: 62.99%, Learning Rate: 0.10%
Epoch 49, Train Loss: 0.2442, Train Accuracy: 93.49%, Validation Loss: 1.9505, Validation Accuracy: 62.04%, Learning Rate: 0.05%
Epoch 50, Train Loss: 0.2198, Train Accuracy: 94.25%, Validation Loss: 1.9837, Validation Accuracy: 62.46%, Learning Rate: 0.01%
Epoch 51, Train Loss: 0.6596, Train Accuracy: 79.75%, Validation Loss: 3.6988, Validation Accuracy: 33.44%, Learning Rate: 0.50%
Epoch 52, Train Loss: 1.5387, Train Accuracy: 53.64%, Validation Loss: 9.4810, Validation Accuracy: 2.00%, Learning Rate: 0.49%
Epoch 53, Train Loss: 1.3354, Train Accuracy: 57.79%, Validation Loss: 2.2164, Validation Accuracy: 47.21%, Learning Rate: 0.45%
Epoch 54, Train Loss: 1.0393, Train Accuracy: 67.42%, Validation Loss: 2.0285, Validation Accuracy: 55.21%, Learning Rate: 0.40%
Epoch 55, Train Loss: 0.8224, Train Accuracy: 74.30%, Validation Loss: 2.2561, Validation Accuracy: 51.52%, Learning Rate: 0.33%
Epoch 56, Train Loss: 0.5903, Train Accuracy: 81.45%, Validation Loss: 5.5662, Validation Accuracy: 14.93%, Learning Rate: 0.25%
Epoch 57, Train Loss: 0.3948, Train Accuracy: 88.55%, Validation Loss: 4.7456, Validation Accuracy: 18.30%, Learning Rate: 0.17%
Epoch 58, Train Loss: 0.3047, Train Accuracy: 91.30%, Validation Loss: 1.9793, Validation Accuracy: 62.57%, Learning Rate: 0.10%
Epoch 59, Train Loss: 0.2165, Train Accuracy: 94.05%, Validation Loss: 1.9665, Validation Accuracy: 62.67%, Learning Rate: 0.05%
Epoch 60, Train Loss: 0.1908, Train Accuracy: 95.23%, Validation Loss: 1.9632, Validation Accuracy: 62.36%, Learning Rate: 0.01%
Epoch 61, Train Loss: 0.5720, Train Accuracy: 82.73%, Validation Loss: 14.5702, Validation Accuracy: 8.73%, Learning Rate: 0.50%
Epoch 62, Train Loss: 1.2915, Train Accuracy: 59.61%, Validation Loss: 2.6566, Validation Accuracy: 41.32%, Learning Rate: 0.49%
Epoch 63, Train Loss: 1.2904, Train Accuracy: 59.93%, Validation Loss: 2.2573, Validation Accuracy: 46.69%, Learning Rate: 0.45%
Epoch 64, Train Loss: 0.9418, Train Accuracy: 70.66%, Validation Loss: 2.3194, Validation Accuracy: 51.31%, Learning Rate: 0.40%
Epoch 65, Train Loss: 0.7819, Train Accuracy: 75.04%, Validation Loss: 2.2865, Validation Accuracy: 54.05%, Learning Rate: 0.33%
Epoch 66, Train Loss: 0.5718, Train Accuracy: 81.84%, Validation Loss: 2.1152, Validation Accuracy: 60.15%, Learning Rate: 0.25%
Epoch 67, Train Loss: 0.3815, Train Accuracy: 88.70%, Validation Loss: 2.0511, Validation Accuracy: 61.41%, Learning Rate: 0.17%
Epoch 68, Train Loss: 0.2676, Train Accuracy: 92.48%, Validation Loss: 2.0931, Validation Accuracy: 62.78%, Learning Rate: 0.10%
Epoch 69, Train Loss: 0.1988, Train Accuracy: 94.45%, Validation Loss: 2.0770, Validation Accuracy: 63.20%, Learning Rate: 0.05%
Epoch 70, Train Loss: 0.1828, Train Accuracy: 95.06%, Validation Loss: 2.0508, Validation Accuracy: 62.88%, Learning Rate: 0.01%
Epoch 71, Train Loss: 0.4387, Train Accuracy: 86.39%, Validation Loss: 2.6482, Validation Accuracy: 48.37%, Learning Rate: 0.50%
Epoch 72, Train Loss: 1.2247, Train Accuracy: 62.60%, Validation Loss: 8.2978, Validation Accuracy: 6.73%, Learning Rate: 0.49%
Epoch 73, Train Loss: 1.3093, Train Accuracy: 60.29%, Validation Loss: 6.4850, Validation Accuracy: 9.15%, Learning Rate: 0.45%
Epoch 74, Train Loss: 0.9702, Train Accuracy: 69.88%, Validation Loss: 2.5752, Validation Accuracy: 46.27%, Learning Rate: 0.40%
Epoch 75, Train Loss: 0.6957, Train Accuracy: 78.35%, Validation Loss: 2.1316, Validation Accuracy: 57.31%, Learning Rate: 0.33%
Epoch 76, Train Loss: 0.5219, Train Accuracy: 83.73%, Validation Loss: 2.0470, Validation Accuracy: 58.25%, Learning Rate: 0.25%
Epoch 77, Train Loss: 0.3522, Train Accuracy: 89.85%, Validation Loss: 2.0711, Validation Accuracy: 61.51%, Learning Rate: 0.17%
Epoch 78, Train Loss: 0.2353, Train Accuracy: 93.49%, Validation Loss: 2.0312, Validation Accuracy: 63.41%, Learning Rate: 0.10%
Epoch 79, Train Loss: 0.1976, Train Accuracy: 94.25%, Validation Loss: 2.0604, Validation Accuracy: 63.62%, Learning Rate: 0.05%
Epoch 80, Train Loss: 0.1776, Train Accuracy: 95.38%, Validation Loss: 2.0569, Validation Accuracy: 63.51%, Learning Rate: 0.01%
Epoch 81, Train Loss: 0.4586, Train Accuracy: 86.02%, Validation Loss: 8.9934, Validation Accuracy: 9.36%, Learning Rate: 0.50%
Epoch 82, Train Loss: 1.2572, Train Accuracy: 62.53%, Validation Loss: 11.3852, Validation Accuracy: 7.36%, Learning Rate: 0.49%
Epoch 83, Train Loss: 1.1860, Train Accuracy: 63.17%, Validation Loss: 2.3386, Validation Accuracy: 45.74%, Learning Rate: 0.45%
Epoch 84, Train Loss: 0.8974, Train Accuracy: 71.30%, Validation Loss: 5.1083, Validation Accuracy: 9.78%, Learning Rate: 0.40%
Epoch 85, Train Loss: 0.6686, Train Accuracy: 79.04%, Validation Loss: 2.0672, Validation Accuracy: 58.68%, Learning Rate: 0.33%
Epoch 86, Train Loss: 0.4640, Train Accuracy: 86.19%, Validation Loss: 2.6122, Validation Accuracy: 50.79%, Learning Rate: 0.25%
Epoch 87, Train Loss: 0.3181, Train Accuracy: 90.29%, Validation Loss: 2.2199, Validation Accuracy: 53.63%, Learning Rate: 0.17%
Epoch 88, Train Loss: 0.2268, Train Accuracy: 93.37%, Validation Loss: 2.0634, Validation Accuracy: 63.09%, Learning Rate: 0.10%
Epoch 89, Train Loss: 0.1769, Train Accuracy: 94.79%, Validation Loss: 2.0426, Validation Accuracy: 63.51%, Learning Rate: 0.05%
Epoch 90, Train Loss: 0.1583, Train Accuracy: 95.48%, Validation Loss: 2.0258, Validation Accuracy: 63.72%, Learning Rate: 0.01%
Epoch 91, Train Loss: 0.3826, Train Accuracy: 88.01%, Validation Loss: 11.7006, Validation Accuracy: 10.09%, Learning Rate: 0.50%
Epoch 92, Train Loss: 1.3109, Train Accuracy: 61.23%, Validation Loss: 10.3495, Validation Accuracy: 5.47%, Learning Rate: 0.49%
Epoch 93, Train Loss: 1.2236, Train Accuracy: 61.74%, Validation Loss: 2.0588, Validation Accuracy: 53.73%, Learning Rate: 0.45%
Epoch 94, Train Loss: 0.9063, Train Accuracy: 71.72%, Validation Loss: 5.1297, Validation Accuracy: 11.15%, Learning Rate: 0.40%
Epoch 95, Train Loss: 0.6722, Train Accuracy: 79.46%, Validation Loss: 2.0176, Validation Accuracy: 57.31%, Learning Rate: 0.33%
Epoch 96, Train Loss: 0.4570, Train Accuracy: 86.46%, Validation Loss: 2.0125, Validation Accuracy: 60.88%, Learning Rate: 0.25%
Epoch 97, Train Loss: 0.3003, Train Accuracy: 91.52%, Validation Loss: 2.1386, Validation Accuracy: 61.09%, Learning Rate: 0.17%
Epoch 98, Train Loss: 0.2211, Train Accuracy: 93.51%, Validation Loss: 2.0366, Validation Accuracy: 64.04%, Learning Rate: 0.10%
Model saved with Validation Accuracy: 64.04%
Epoch 99, Train Loss: 0.1661, Train Accuracy: 95.50%, Validation Loss: 2.0732, Validation Accuracy: 63.30%, Learning Rate: 0.05%
Epoch 100, Train Loss: 0.1627, Train Accuracy: 95.50%, Validation Loss: 2.0866, Validation Accuracy: 62.99%, Learning Rate: 0.01%
Epoch 101, Train Loss: 0.3998, Train Accuracy: 87.62%, Validation Loss: 2.7482, Validation Accuracy: 48.69%, Learning Rate: 0.50%
Epoch 102, Train Loss: 1.1540, Train Accuracy: 64.79%, Validation Loss: 6.4551, Validation Accuracy: 9.25%, Learning Rate: 0.49%
Epoch 103, Train Loss: 1.1474, Train Accuracy: 64.37%, Validation Loss: 10.7821, Validation Accuracy: 8.52%, Learning Rate: 0.45%
Epoch 104, Train Loss: 0.7873, Train Accuracy: 76.17%, Validation Loss: 2.1819, Validation Accuracy: 56.89%, Learning Rate: 0.40%
Epoch 105, Train Loss: 0.6064, Train Accuracy: 81.23%, Validation Loss: 2.0777, Validation Accuracy: 58.36%, Learning Rate: 0.33%
Epoch 106, Train Loss: 0.4247, Train Accuracy: 87.35%, Validation Loss: 2.0624, Validation Accuracy: 60.99%, Learning Rate: 0.25%
Epoch 107, Train Loss: 0.2913, Train Accuracy: 91.43%, Validation Loss: 5.5841, Validation Accuracy: 15.46%, Learning Rate: 0.17%
Epoch 108, Train Loss: 0.1762, Train Accuracy: 95.21%, Validation Loss: 2.5248, Validation Accuracy: 48.79%, Learning Rate: 0.10%
Epoch 109, Train Loss: 0.1585, Train Accuracy: 95.58%, Validation Loss: 2.0594, Validation Accuracy: 63.30%, Learning Rate: 0.05%
Epoch 110, Train Loss: 0.1398, Train Accuracy: 96.02%, Validation Loss: 2.0695, Validation Accuracy: 64.14%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 64.14%
Epoch 111, Train Loss: 0.4282, Train Accuracy: 86.93%, Validation Loss: 6.5868, Validation Accuracy: 12.30%, Learning Rate: 0.50%
Epoch 112, Train Loss: 1.2478, Train Accuracy: 61.62%, Validation Loss: 8.3507, Validation Accuracy: 6.41%, Learning Rate: 0.49%
Epoch 113, Train Loss: 1.2411, Train Accuracy: 61.82%, Validation Loss: 2.1824, Validation Accuracy: 51.31%, Learning Rate: 0.45%
Epoch 114, Train Loss: 0.9071, Train Accuracy: 72.26%, Validation Loss: 2.2339, Validation Accuracy: 50.68%, Learning Rate: 0.40%
Epoch 115, Train Loss: 0.6145, Train Accuracy: 81.13%, Validation Loss: 2.1071, Validation Accuracy: 60.25%, Learning Rate: 0.33%
Epoch 116, Train Loss: 0.4151, Train Accuracy: 87.15%, Validation Loss: 2.2847, Validation Accuracy: 60.15%, Learning Rate: 0.25%
Epoch 117, Train Loss: 0.2786, Train Accuracy: 91.77%, Validation Loss: 2.1494, Validation Accuracy: 60.25%, Learning Rate: 0.17%
Epoch 118, Train Loss: 0.1978, Train Accuracy: 94.57%, Validation Loss: 2.0365, Validation Accuracy: 63.09%, Learning Rate: 0.10%
Epoch 119, Train Loss: 0.1565, Train Accuracy: 95.85%, Validation Loss: 2.0890, Validation Accuracy: 62.46%, Learning Rate: 0.05%
Epoch 120, Train Loss: 0.1428, Train Accuracy: 95.97%, Validation Loss: 2.0861, Validation Accuracy: 62.67%, Learning Rate: 0.01%
Epoch 121, Train Loss: 0.3919, Train Accuracy: 88.08%, Validation Loss: 5.2155, Validation Accuracy: 12.83%, Learning Rate: 0.50%
Epoch 122, Train Loss: 1.0838, Train Accuracy: 65.82%, Validation Loss: 8.7766, Validation Accuracy: 8.20%, Learning Rate: 0.49%
Epoch 123, Train Loss: 1.0796, Train Accuracy: 65.70%, Validation Loss: 8.7501, Validation Accuracy: 9.15%, Learning Rate: 0.45%
Epoch 124, Train Loss: 0.8421, Train Accuracy: 73.46%, Validation Loss: 2.0915, Validation Accuracy: 55.31%, Learning Rate: 0.40%
Epoch 125, Train Loss: 0.6019, Train Accuracy: 81.67%, Validation Loss: 2.1512, Validation Accuracy: 60.04%, Learning Rate: 0.33%
Epoch 126, Train Loss: 0.3685, Train Accuracy: 89.14%, Validation Loss: 2.1413, Validation Accuracy: 61.41%, Learning Rate: 0.25%
Epoch 127, Train Loss: 0.2494, Train Accuracy: 92.70%, Validation Loss: 2.1651, Validation Accuracy: 62.04%, Learning Rate: 0.17%
Epoch 128, Train Loss: 0.1744, Train Accuracy: 95.41%, Validation Loss: 2.1301, Validation Accuracy: 62.57%, Learning Rate: 0.10%
Epoch 129, Train Loss: 0.1425, Train Accuracy: 96.49%, Validation Loss: 2.1673, Validation Accuracy: 63.09%, Learning Rate: 0.05%
Epoch 130, Train Loss: 0.1239, Train Accuracy: 96.58%, Validation Loss: 2.1605, Validation Accuracy: 63.20%, Learning Rate: 0.01%
Epoch 131, Train Loss: 0.3124, Train Accuracy: 90.44%, Validation Loss: 2.5950, Validation Accuracy: 52.26%, Learning Rate: 0.50%
Epoch 132, Train Loss: 0.9210, Train Accuracy: 71.45%, Validation Loss: 9.0433, Validation Accuracy: 8.20%, Learning Rate: 0.49%
Epoch 133, Train Loss: 1.0649, Train Accuracy: 67.42%, Validation Loss: 5.7985, Validation Accuracy: 13.04%, Learning Rate: 0.45%
Epoch 134, Train Loss: 0.7612, Train Accuracy: 77.00%, Validation Loss: 6.7273, Validation Accuracy: 9.25%, Learning Rate: 0.40%
Epoch 135, Train Loss: 0.5685, Train Accuracy: 82.83%, Validation Loss: 4.2528, Validation Accuracy: 20.08%, Learning Rate: 0.33%
Epoch 136, Train Loss: 0.3772, Train Accuracy: 88.57%, Validation Loss: 2.1928, Validation Accuracy: 61.83%, Learning Rate: 0.25%
Epoch 137, Train Loss: 0.2411, Train Accuracy: 92.51%, Validation Loss: 2.0765, Validation Accuracy: 62.57%, Learning Rate: 0.17%
Epoch 138, Train Loss: 0.1741, Train Accuracy: 95.31%, Validation Loss: 2.0770, Validation Accuracy: 62.57%, Learning Rate: 0.10%
Epoch 139, Train Loss: 0.1415, Train Accuracy: 95.90%, Validation Loss: 2.0918, Validation Accuracy: 62.99%, Learning Rate: 0.05%
Epoch 140, Train Loss: 0.1141, Train Accuracy: 96.88%, Validation Loss: 2.0800, Validation Accuracy: 63.41%, Learning Rate: 0.01%
Epoch 141, Train Loss: 0.3190, Train Accuracy: 90.15%, Validation Loss: 11.1988, Validation Accuracy: 10.62%, Learning Rate: 0.50%
Epoch 142, Train Loss: 0.9544, Train Accuracy: 71.65%, Validation Loss: 7.0254, Validation Accuracy: 9.25%, Learning Rate: 0.49%
Epoch 143, Train Loss: 1.1003, Train Accuracy: 66.09%, Validation Loss: 2.3984, Validation Accuracy: 46.06%, Learning Rate: 0.45%
Epoch 144, Train Loss: 0.7721, Train Accuracy: 76.09%, Validation Loss: 2.3662, Validation Accuracy: 54.57%, Learning Rate: 0.40%
Epoch 145, Train Loss: 0.5554, Train Accuracy: 82.73%, Validation Loss: 2.1279, Validation Accuracy: 55.73%, Learning Rate: 0.33%
Epoch 146, Train Loss: 0.3726, Train Accuracy: 88.53%, Validation Loss: 2.2298, Validation Accuracy: 59.83%, Learning Rate: 0.25%
Epoch 147, Train Loss: 0.2352, Train Accuracy: 92.92%, Validation Loss: 2.1352, Validation Accuracy: 61.51%, Learning Rate: 0.17%
Epoch 148, Train Loss: 0.1789, Train Accuracy: 95.16%, Validation Loss: 2.1628, Validation Accuracy: 62.36%, Learning Rate: 0.10%
Epoch 149, Train Loss: 0.1184, Train Accuracy: 96.51%, Validation Loss: 2.1587, Validation Accuracy: 63.20%, Learning Rate: 0.05%
Epoch 150, Train Loss: 0.1205, Train Accuracy: 96.73%, Validation Loss: 2.1476, Validation Accuracy: 63.20%, Learning Rate: 0.01%
Epoch 151, Train Loss: 0.2788, Train Accuracy: 91.94%, Validation Loss: 2.7563, Validation Accuracy: 52.58%, Learning Rate: 0.50%
Epoch 152, Train Loss: 0.7569, Train Accuracy: 76.58%, Validation Loss: 9.9319, Validation Accuracy: 6.52%, Learning Rate: 0.49%
Epoch 153, Train Loss: 0.9821, Train Accuracy: 69.34%, Validation Loss: 2.3844, Validation Accuracy: 49.63%, Learning Rate: 0.45%
Epoch 154, Train Loss: 0.8075, Train Accuracy: 75.31%, Validation Loss: 2.3838, Validation Accuracy: 55.21%, Learning Rate: 0.40%
Epoch 155, Train Loss: 0.5914, Train Accuracy: 81.57%, Validation Loss: 7.5986, Validation Accuracy: 10.09%, Learning Rate: 0.33%
Epoch 156, Train Loss: 0.3626, Train Accuracy: 89.14%, Validation Loss: 2.1929, Validation Accuracy: 58.99%, Learning Rate: 0.25%
Epoch 157, Train Loss: 0.2254, Train Accuracy: 93.44%, Validation Loss: 2.1452, Validation Accuracy: 62.15%, Learning Rate: 0.17%
Epoch 158, Train Loss: 0.1558, Train Accuracy: 95.50%, Validation Loss: 2.1095, Validation Accuracy: 61.93%, Learning Rate: 0.10%
Epoch 159, Train Loss: 0.1162, Train Accuracy: 96.95%, Validation Loss: 2.1030, Validation Accuracy: 62.57%, Learning Rate: 0.05%
Epoch 160, Train Loss: 0.0973, Train Accuracy: 97.35%, Validation Loss: 2.1146, Validation Accuracy: 62.36%, Learning Rate: 0.01%
Epoch 161, Train Loss: 0.2772, Train Accuracy: 91.74%, Validation Loss: 5.6835, Validation Accuracy: 15.56%, Learning Rate: 0.50%
Epoch 162, Train Loss: 0.8425, Train Accuracy: 74.15%, Validation Loss: 5.2124, Validation Accuracy: 18.09%, Learning Rate: 0.49%
Epoch 163, Train Loss: 0.9530, Train Accuracy: 70.42%, Validation Loss: 5.7160, Validation Accuracy: 18.40%, Learning Rate: 0.45%
Epoch 164, Train Loss: 0.7466, Train Accuracy: 76.71%, Validation Loss: 4.5811, Validation Accuracy: 16.82%, Learning Rate: 0.40%
Epoch 165, Train Loss: 0.4870, Train Accuracy: 84.84%, Validation Loss: 2.0713, Validation Accuracy: 61.83%, Learning Rate: 0.33%
Epoch 166, Train Loss: 0.3305, Train Accuracy: 90.27%, Validation Loss: 9.2305, Validation Accuracy: 9.36%, Learning Rate: 0.25%
Epoch 167, Train Loss: 0.2140, Train Accuracy: 93.88%, Validation Loss: 2.1899, Validation Accuracy: 61.93%, Learning Rate: 0.17%
Epoch 168, Train Loss: 0.1445, Train Accuracy: 95.95%, Validation Loss: 2.1130, Validation Accuracy: 63.62%, Learning Rate: 0.10%
Epoch 169, Train Loss: 0.1087, Train Accuracy: 97.20%, Validation Loss: 2.1448, Validation Accuracy: 63.41%, Learning Rate: 0.05%
Epoch 170, Train Loss: 0.1083, Train Accuracy: 96.90%, Validation Loss: 2.1348, Validation Accuracy: 63.20%, Learning Rate: 0.01%
Epoch 171, Train Loss: 0.2361, Train Accuracy: 93.17%, Validation Loss: 2.7857, Validation Accuracy: 58.36%, Learning Rate: 0.50%
Epoch 172, Train Loss: 0.6862, Train Accuracy: 78.60%, Validation Loss: 3.6622, Validation Accuracy: 33.54%, Learning Rate: 0.49%
Epoch 173, Train Loss: 0.9750, Train Accuracy: 69.95%, Validation Loss: 2.6617, Validation Accuracy: 44.58%, Learning Rate: 0.45%
Epoch 174, Train Loss: 0.7457, Train Accuracy: 76.73%, Validation Loss: 3.0731, Validation Accuracy: 41.01%, Learning Rate: 0.40%
Epoch 175, Train Loss: 0.5324, Train Accuracy: 83.24%, Validation Loss: 2.1683, Validation Accuracy: 59.31%, Learning Rate: 0.33%
Epoch 176, Train Loss: 0.3431, Train Accuracy: 89.75%, Validation Loss: 2.2594, Validation Accuracy: 61.41%, Learning Rate: 0.25%
Epoch 177, Train Loss: 0.2218, Train Accuracy: 93.39%, Validation Loss: 2.2977, Validation Accuracy: 62.25%, Learning Rate: 0.17%
Epoch 178, Train Loss: 0.1431, Train Accuracy: 96.04%, Validation Loss: 2.1874, Validation Accuracy: 63.09%, Learning Rate: 0.10%
Epoch 179, Train Loss: 0.1152, Train Accuracy: 96.76%, Validation Loss: 2.1867, Validation Accuracy: 62.99%, Learning Rate: 0.05%
Epoch 180, Train Loss: 0.0958, Train Accuracy: 97.59%, Validation Loss: 2.1838, Validation Accuracy: 62.78%, Learning Rate: 0.01%
Epoch 181, Train Loss: 0.2330, Train Accuracy: 93.05%, Validation Loss: 2.8395, Validation Accuracy: 56.68%, Learning Rate: 0.50%
Epoch 182, Train Loss: 0.7953, Train Accuracy: 75.82%, Validation Loss: 4.4324, Validation Accuracy: 17.25%, Learning Rate: 0.49%
Epoch 183, Train Loss: 0.8989, Train Accuracy: 72.21%, Validation Loss: 3.2423, Validation Accuracy: 21.87%, Learning Rate: 0.45%
Epoch 184, Train Loss: 0.7410, Train Accuracy: 76.95%, Validation Loss: 2.3929, Validation Accuracy: 52.58%, Learning Rate: 0.40%
Epoch 185, Train Loss: 0.4758, Train Accuracy: 84.82%, Validation Loss: 2.2530, Validation Accuracy: 60.57%, Learning Rate: 0.33%
Epoch 186, Train Loss: 0.3243, Train Accuracy: 89.68%, Validation Loss: 2.2728, Validation Accuracy: 61.30%, Learning Rate: 0.25%
Epoch 187, Train Loss: 0.2113, Train Accuracy: 93.49%, Validation Loss: 2.2645, Validation Accuracy: 61.72%, Learning Rate: 0.17%
Epoch 188, Train Loss: 0.1393, Train Accuracy: 96.49%, Validation Loss: 2.2233, Validation Accuracy: 62.25%, Learning Rate: 0.10%
Epoch 189, Train Loss: 0.1089, Train Accuracy: 96.78%, Validation Loss: 2.2134, Validation Accuracy: 62.67%, Learning Rate: 0.05%
Epoch 190, Train Loss: 0.1020, Train Accuracy: 97.27%, Validation Loss: 2.2099, Validation Accuracy: 62.57%, Learning Rate: 0.01%
Epoch 191, Train Loss: 0.2585, Train Accuracy: 91.79%, Validation Loss: 6.3546, Validation Accuracy: 11.67%, Learning Rate: 0.50%
Epoch 192, Train Loss: 0.8559, Train Accuracy: 74.15%, Validation Loss: 6.7039, Validation Accuracy: 9.15%, Learning Rate: 0.49%
Epoch 193, Train Loss: 0.9579, Train Accuracy: 70.00%, Validation Loss: 4.7026, Validation Accuracy: 16.72%, Learning Rate: 0.45%
Epoch 194, Train Loss: 0.6836, Train Accuracy: 79.29%, Validation Loss: 2.3651, Validation Accuracy: 53.31%, Learning Rate: 0.40%
Epoch 195, Train Loss: 0.4732, Train Accuracy: 86.00%, Validation Loss: 2.3155, Validation Accuracy: 57.83%, Learning Rate: 0.33%
Epoch 196, Train Loss: 0.3259, Train Accuracy: 90.27%, Validation Loss: 2.2987, Validation Accuracy: 62.04%, Learning Rate: 0.25%
Epoch 197, Train Loss: 0.2036, Train Accuracy: 94.10%, Validation Loss: 2.1233, Validation Accuracy: 61.41%, Learning Rate: 0.17%
Epoch 198, Train Loss: 0.1241, Train Accuracy: 96.54%, Validation Loss: 2.3204, Validation Accuracy: 62.04%, Learning Rate: 0.10%
Epoch 199, Train Loss: 0.1063, Train Accuracy: 97.15%, Validation Loss: 2.2339, Validation Accuracy: 62.46%, Learning Rate: 0.05%
Epoch 200, Train Loss: 0.0907, Train Accuracy: 97.49%, Validation Loss: 2.2450, Validation Accuracy: 62.57%, Learning Rate: 0.01%
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<ipython-input-7-fa98badd98ed> in <cell line: 194>()
    192
    193 # 모델 학습 시작
--> 194 train_model(train_loader, val_loader, model, optimizer, criterion, scheduler, epochs=300, save_path='/content/drive/MyDrive/진킴/model_final5.pth')

6 frames
/usr/local/lib/python3.10/dist-packages/PIL/Image.py in open(fp, mode, formats)
   3429
   3430     if filename:
-> 3431         fp = builtins.open(filename, "rb")
   3432         exclusive_fp = True
   3433     else:

KeyboardInterrupt:


## 검증 정확도 최고결과

Epoch 110, Train Loss: 0.1398, Train Accuracy: 96.02%, Validation Loss: 2.0695, Validation Accuracy: 64.14%, Learning Rate: 0.01%
Model saved with Validation Accuracy: 64.14%

사실상 320 에폭.



---



##[2-1]

In [ ]:
!pip install ujson

In [ ]:
# 모든 행과 열을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# 이미지와 JSON 파일의 경로
image_dir = '/content/drive/MyDrive/dataset/'
training_image_dir = os.path.join(image_dir, 'training_image')
validation_image_dir = os.path.join(image_dir, 'validation_image')
training_label_dir = os.path.join(image_dir, 'training_label')
validation_label_dir = os.path.join(image_dir, 'validation_label')

# 이미지 파일과 JSON 파일의 패턴 정의
image_pattern = re.compile(r"^(W|T)_(\d+)_(\d+)_(\w+)_(\w)\.jpg$")
label_pattern = re.compile(r"^(W|T)_(\d+)_(\d+)_(\w+)_(\w)_(\d+)\.json$")

# 유효한 이미지 ID를 저장할 집합
def get_valid_image_ids(image_dir):
    valid_image_ids = set()
    for filename in os.listdir(image_dir):
        if image_pattern.match(filename):
            img_id = image_pattern.match(filename).group(2)
            valid_image_ids.add(img_id)
    return valid_image_ids

# 통계 계산 함수 (Q5 포함)
def calculate_statistics_with_q5(label_dir, valid_image_ids):
    stats = []
    valid_labels = []

    for filename in os.listdir(label_dir):
        match = label_pattern.match(filename)
        if match:
            img_id = match.group(2)
            style = match.group(4)
            gender = "여성" if match.group(1) == "W" else "남성"
            if img_id in valid_image_ids:
                with open(os.path.join(label_dir, filename), 'r') as f:
                    data = json.load(f)
                    # 통계에 사용할 정보 추가
                    stats.append({"성별": gender, "스타일": style, "이미지 수": 1})
                    # Q5 포함하여 유효한 라벨 저장
                    valid_labels.append({
                        "respondent_id": data["user"]["R_id"],
                        "gender": gender,
                        "style": style,
                        "image_id": img_id,
                        "Q5": data["item"]["survey"]["Q5"],
                        "img_name": data["item"]["imgName"]
                    })

    # 통계 데이터프레임 생성 및 집계
    stats_df = pd.DataFrame(stats)
    stats_df = stats_df.groupby(["성별", "스타일"]).sum().reset_index()
    return stats_df, valid_labels

# 유효한 이미지 ID 확인
valid_training_ids = get_valid_image_ids(training_image_dir)
valid_validation_ids = get_valid_image_ids(validation_image_dir)

# 통계 테이블 및 유효 라벨 데이터 생성
training_stats, valid_training_labels = calculate_statistics_with_q5(training_label_dir, valid_training_ids)
validation_stats, valid_validation_labels = calculate_statistics_with_q5(validation_label_dir, valid_validation_ids)

# 출력
print("Training 통계표:")
display(training_stats)
print("\nValidation 통계표:")
display(validation_stats)

Training 통계표:


,성별,스타일,이미지 수
0,남성,athleisure,2
1,남성,bodyconscious,10
2,남성,bold,18
3,남성,cityglam,11
4,남성,classic,21
5,남성,disco,1
6,남성,ecology,9
7,남성,genderless,18
8,남성,grunge,3
9,남성,hiphop,64



Validation 통계표:


,성별,스타일,이미지 수
0,남성,bodyconscious,1
1,남성,bold,3
2,남성,cityglam,2
3,남성,classic,5
4,남성,ecology,1
5,남성,hiphop,3
6,남성,hippie,1
7,남성,ivy,3
8,남성,mods,1
9,남성,normcore,4




---



##[2-2] [2-1]도 출력 같이 나옴. 사실상 이 코드가 미션 2 자체

In [ ]:
import os
import re
import json
from collections import defaultdict, Counter
import pandas as pd

# 데이터 디렉터리 경로
train_image_dir = '/content/drive/MyDrive/dataset/training_image'
val_image_dir = '/content/drive/MyDrive/dataset/validation_image'

train_label_dir = '/content/drive/MyDrive/dataset/training_label'
val_label_dir = '/content/drive/MyDrive/dataset/validation_label'

# 정규 표현식 패턴 정의
image_pattern = re.compile(r'^(W|T)_(\d+)_(.*?)_(.*?)_(W|M)\.jpg$')
label_pattern = re.compile(r'^(W|T)_(\d+)_(.*?)_(.*?)_(W|M)_(\d+)\.json$')

# 유효한 이미지 ID 가져오는 함수
def get_valid_image_ids(image_dir):
    image_ids = set()
    for filename in os.listdir(image_dir):
        match = image_pattern.match(filename)
        if match:
            W_T, image_id, era, style, gender = match.groups()
            image_ids.add(image_id)
    return image_ids

# 라벨 데이터를 처리하는 함수
def process_labeling_data(label_dir, valid_image_ids, dataset_name):
    stats = defaultdict(int)
    respondent_counts = defaultdict(int)
    style_preferences = defaultdict(lambda: defaultdict(list))  # R_id -> {'선호 스타일': [파일명], ...}
    all_r_ids = set()

    for filename in os.listdir(label_dir):
        match = label_pattern.match(filename)
        if match:
            W_T, image_id, era, style, gender, survey_id = match.groups()

            # 유효한 이미지 ID만 처리
            if image_id not in valid_image_ids:
                continue

            # 성별 매핑 ('W'는 '여성', 'M'은 '남성')
            if gender == 'W':
                gender_label = 'Female'
            elif gender == 'M':
                gender_label = 'Male'
            else:
                continue

            label = f"{gender_label}_{style}"

            # 성별 및 스타일별 조사 ID 카운팅
            stats[label] += 1

            # JSON 파일 읽기
            json_path = os.path.join(label_dir, filename)
            with open(json_path, 'r') as f:
                data = json.load(f)

            # 응답자 ID 및 스타일 선호도 가져오기
            R_id = data['user']['R_id']
            Q5 = data['item']['survey'].get('Q5', None)

            # 응답자 정보 수집
            respondent_counts[R_id] += 1
            all_r_ids.add(R_id)

            if Q5 is not None:
                preference = 'Preferred' if Q5 == 2 else 'Not Preferred' if Q5 == 1 else 'Unknown'
                if preference in ['Preferred', 'Not Preferred']:
                    # 선호 여부와 연관된 파일명 저장
                    full_filename = data['imgName']  # 'imgName'이 JSON에 있다고 가정
                    key = f"{dataset_name} Style {preference}"
                    style_preferences[R_id][key].append(full_filename)

    return stats, respondent_counts, style_preferences, all_r_ids

# 통계 테이블 생성 함수
def create_stats_table(stats):
    # 성별과 스타일 추출
    gender_style_counts = defaultdict(lambda: defaultdict(int))
    for label, count in stats.items():
        gender, style = label.split('_', 1)
        gender_style_counts[gender][style] += count

    # DataFrame 생성
    df = pd.DataFrame(gender_style_counts).fillna(0).astype(int).sort_index()
    df = df.transpose()
    df['Total'] = df.sum(axis=1)
    df.loc['Total'] = df.sum()

    return df

# 학습 데이터 처리
print("학습 데이터 처리 중...")
train_valid_image_ids = get_valid_image_ids(train_image_dir)
train_stats, train_respondent_counts, train_style_preferences, train_r_ids = process_labeling_data(
    train_label_dir, train_valid_image_ids, 'Training')

train_stats_table = create_stats_table(train_stats)
print("학습 데이터 통계:")
print(train_stats_table)

# 검증 데이터 처리
print("\n검증 데이터 처리 중...")
val_valid_image_ids = get_valid_image_ids(val_image_dir)
val_stats, val_respondent_counts, val_style_preferences, val_r_ids = process_labeling_data(
    val_label_dir, val_valid_image_ids, 'Validation')

val_stats_table = create_stats_table(val_stats)
print("검증 데이터 통계:")
print(val_stats_table)

# 모든 응답자의 스타일 선호도를 결합
combined_style_preferences = defaultdict(lambda: defaultdict(list))

all_r_ids = train_style_preferences.keys() | val_style_preferences.keys()

for R_id in all_r_ids:
    # 학습 데이터 선호도 병합
    for key, filenames in train_style_preferences.get(R_id, {}).items():
        combined_style_preferences[R_id][key].extend(filenames)
    # 검증 데이터 선호도 병합
    for key, filenames in val_style_preferences.get(R_id, {}).items():
        combined_style_preferences[R_id][key].extend(filenames)

# 응답자 수를 합산
total_respondent_counts = Counter()
for R_id in all_r_ids:
    total_responses = train_respondent_counts.get(R_id, 0) + val_respondent_counts.get(R_id, 0)
    total_respondent_counts[R_id] = total_responses

# 총 응답 수를 기준으로 상위 100명 추출
top_respondents = total_respondent_counts.most_common(100)
top_r_ids = [R_id for R_id, count in top_respondents]

# DataFrame 생성을 위한 데이터 준비
rows = []
for R_id in top_r_ids:
    preferences = combined_style_preferences.get(R_id, {})
    row = {
        'Respondent ID': R_id,
        'Training Style Preferred': ', '.join(preferences.get('Training Style Preferred', [])),
        'Training Style Not Preferred': ', '.join(preferences.get('Training Style Not Preferred', [])),
        'Validation Style Preferred': ', '.join(preferences.get('Validation Style Preferred', [])),
        'Validation Style Not Preferred': ', '.join(preferences.get('Validation Style Not Preferred', [])),
    }
    rows.append(row)

# 상위 100명의 응답자 데이터를 DataFrame으로 생성
top_preference_df = pd.DataFrame(rows)

# DataFrame을 Excel 파일로 저장
output_excel_path = '/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/top_100_respondents_preferences.xlsx'
top_preference_df.to_excel(output_excel_path, index=False, engine='openpyxl')
print(f"\n상위 100명의 응답자 선호도가 {output_excel_path}에 저장되었습니다.")


# DataFrame 출력
print("\n상위 100명의 응답자 스타일 선호도 정보 테이블:")
print(top_preference_df.head())


학습 데이터 처리 중...
학습 데이터 통계:
        athleisure  bodyconscious  bold  cityglam  classic  disco  ecology  \
Female          67             81     0        51       75     32       45   
Male             0              0   198         0        0      0        0   
Total           67             81   198        51       75     32       45   

        feminine  genderless  grunge  hiphop  hippie  ivy  kitsch  lingerie  \
Female       144          51      19      50      62    0      61        33   
Male           0           0       0     230     273  335       0         0   
Total        144          51      19     280     335  335      61        33   

        lounge  metrosexual  military  minimal  mods  normcore  oriental  \
Female      32            0        21      131     0       119        71   
Male         0          218         0        0   302       200         0   
Total       32          218        21      131   302       319        71   

        popart  powersuit  punk  space 



---



##[3-1]

[3-1]

### 정의
 - 아이템 기반 필터링 (Item-based Filtering): 사용자들이 선호하는 항목 간의 유사성을 분석하여, 사용자가 좋아하는 항목과 유사한 다른 항목을 추천하는 방법이다. 이 방식은 사용자들 간의 유사성보다는 아이템 간의 관계에 초점을 맞춘다.

 - 사용자 기반 필터링 (User-based Filtering): 비슷한 취향을 가진 사용자들 간의 관계를 분석하여, 유사한 사용자가 선호하는 항목을 추천하는 방법이다. 이 방식은 사용자의과거 선호도와 다른 사용자의 행동을 기반으로 추천을 생성한다.  

 #### 1.아이템 기반 필터링
 ##### 적용방법
  - 유사성 측정: 사용자가 선호한 스타일(2)을 바탕으로 스타일 간의 유사성을 계산한다.예를 들어, 코사인 유사도(cosine similarity) 또는 자카드 유사도(Jaccardsimilarity) 등을 사용하여 스타일 간의 관계를 분석한다.
  
  - 추천 생성: 사용자가 선호하는 스타일과 유사한 다른 스타일을 추천한다. 사용자가 A 스타일을 선호한다면, A 스타일과 유사한 B, C, D 스타일을 추천할 수 있다.

 ##### 장점
  - 사용자의 자신의 평가 데이터를 활용하여 추천을 수행하기 때문에 더 관련성 높은 추천제공한다.
   
  - 다양한 항목을 추천할 가능성이 높아 추천 목록의 다양성 더 커진다.

  - 비선호 스타일에 대한 정보가 부족하더라도, 이미 선호한 스타일을 기반으로 추천이 이루어지므로 추천의 질이 비교적 안정적이다.

  - 스타일의 특성을 분석하여 유사한 스타일을 추천함으로써, 사용자의 취향을 잘 반영할 수 있다.

  - 기존 스타일과의 유사성을 활용하여 새로운 스타일을 추천할 수있는 유연성을 가진다.

   ##### 단점
   - 이웃 사용자의 정보가 익명으로 처리되기 때문에 설명하기 어렵다.

   - 스타일 간의 유사성을 제대로 분석하지 못하면 추천의 품질이떨어질 수 있다.

   #### 2.사용자 기반 필터링
   ##### 적용방법
   - 유사 사용자 측정: 사용자의 선호(2)와 비선호(1) 데이터를 바탕으로 유사한 사용자를 찾는다. 이때, 사용자의 비선호 정보도 포함하여 유사성을 분석한다.

   - 추천 생성: 유사한 사용자가 선호하는 스타일을 추천한다. 예를 들어, 사용자가 B 스타일을 비선호하는 경우, 그와 비슷한 사용자가 선호하는 다른 스타일을 추천할 수 있다.

   ##### 장점
   - 사용자의 취향을 반영한 추천이 가능하여, 개인화된 경험을 제공할 수 있다. → 추천의 이유를 명확하게 설명할 수 있다.

   - 비슷한 취향을 가진 사용자들 간의 추천을 통해 다양한 스타일을추천할 수 있다.

   ##### 단점
   - 비선호(1) 스타일에 대한 정보가 많지 않으면, 유사한 사용자를 찾기 어려워질 수 있으며, 이는 추천의 질 저하로 이어질 수 있다.

   - 사용자 수가 적거나 비선호 데이터가 부족할 경우, 유사한 사용자를 찾기가 힘들어질 수 있다. 이는 추천의 신뢰성을 떨어뜨릴 수 있다.

   - 다른 사용자의 평가를 바탕으로 추천이 이루어지므로 정확도가 떨어질 수 있다.



---



##[3-2]

##torchvision 충돌시 sympy 충돌 오류 방지

In [ ]:
!pip install -U sympy

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

# 경로 설정
train_image_dir = '/content/drive/MyDrive/dataset/processed_segmentation_cleaned'
val_image_dir = '/content/drive/MyDrive/dataset/processed_segmentation_cleaned_for_val'
model_path = '/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/model_final5.pth'
fixed_excel_file_path = '/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/top_100_respondents_preferences_fixed.xlsx'

# ResNet 특징 추출기 클래스 정의 (fc 레이어 제외)
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, model_path):
        super(ResNetFeatureExtractor, self).__init__()
        resnet = models.resnet18(pretrained=False)
        state_dict = torch.load(model_path)
        state_dict = {k: v for k, v in state_dict.items() if not k.startswith("fc.")}
        resnet.load_state_dict(state_dict, strict=False)
        self.features = nn.Sequential(*list(resnet.children())[:-2])

    def forward(self, x):
        return self.features(x)

# 이미지 전처리 파이프라인 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

# 데이터셋에서 이미지들의 특징 벡터를 추출하는 함수
def extract_features_for_dataset(image_paths, feature_extractor, device, log_missing_files=False):
    features = []
    feature_extractor.eval()
    missing_files = []

    with torch.no_grad():
        for image_path in tqdm(image_paths, desc="Extracting Features"):
            if os.path.exists(image_path):
                image_tensor = load_and_preprocess_image(image_path, device)
                feature_vector = feature_extractor(image_tensor).squeeze().cpu().numpy()
                features.append(feature_vector)
            else:
                missing_files.append(image_path)
                print(f"File not found: {image_path}")

    if log_missing_files:
        with open("missing_files_log.txt", "w") as file:
            for item in missing_files:
                file.write(f"{item}\n")
        print(f"Missing files logged in 'missing_files_log.txt'")

    return np.array(features)

# 코사인 유사도 및 유클리디안 거리 계산 함수 (배치 처리 적용)
def calculate_similarity_scores(val_features, train_features, batch_size=100):
    cosine_similarity_list = []
    euclidean_distance_list = []

    val_features = torch.tensor(val_features).to(device)
    train_features = torch.tensor(train_features).to(device)
    val_features = val_features.view(val_features.size(0), -1)
    train_features = train_features.view(train_features.size(0), -1)

    with torch.no_grad():
        for i in range(0, len(val_features), batch_size):
            val_batch = val_features[i:i + batch_size]

            # 코사인 유사도 계산
            val_norm = val_batch / val_batch.norm(dim=1, keepdim=True)
            train_norm = train_features / train_features.norm(dim=1, keepdim=True)
            cosine_sim_batch = torch.mm(val_norm, train_norm.T).cpu().numpy()
            cosine_similarity_list.append(cosine_sim_batch)

            # 유클리디안 거리 계산
            val_batch_cpu = val_batch.cpu().numpy()
            train_features_cpu = train_features.cpu().numpy()
            euclidean_dist_batch = -np.linalg.norm(val_batch_cpu[:, None, :] - train_features_cpu[None, :, :], axis=2)
            euclidean_distance_list.append(euclidean_dist_batch)

    cosine_similarity = np.vstack(cosine_similarity_list)
    euclidean_distance = np.vstack(euclidean_distance_list)

    return cosine_similarity, euclidean_distance

# 두 유사도를 결합하는 함수
def combine_similarity_scores(cosine_similarity, euclidean_distance, alpha):
    return alpha * cosine_similarity + (1 - alpha) * euclidean_distance

# 최적 alpha 찾기
def find_optimal_alpha(cosine_similarity, euclidean_distance, train_preferences, val_labels):
    best_alpha, best_f1 = 0.5, 0
    for alpha in np.arange(0.1, 1.0, 0.1):
        combined_score = combine_similarity_scores(cosine_similarity, euclidean_distance, alpha)
        predicted_preferences = predict_style_preference(combined_score, train_preferences)
        _, _, _, f1 = calculate_metrics(predicted_preferences, val_labels)
        if f1 > best_f1:
            best_f1, best_alpha = f1, alpha
    return best_alpha

# 최적 threshold 찾기
def find_optimal_threshold(similarity_scores, train_preferences, val_labels):
    best_threshold, best_f1 = 0.5, 0
    for threshold in np.arange(0.1, 0.9, 0.01):
        predicted_preferences = predict_style_preference(similarity_scores, train_preferences, threshold)
        _, _, _, f1 = calculate_metrics(predicted_preferences, val_labels)
        if f1 > best_f1:
            best_f1, best_threshold = f1, threshold
    return best_threshold

# 스타일 선호도 예측
def predict_style_preference(similarity_scores, train_preferences, threshold=0.5):
    predicted_preferences = []
    train_len = len(train_preferences)

    for score_row in similarity_scores:
        max_similarity = np.max(score_row)
        most_similar_index = np.argmax(score_row) if np.argmax(score_row) < train_len else train_len - 1
        predicted_label = train_preferences[most_similar_index] if max_similarity > threshold else 0
        predicted_preferences.append(predicted_label)

    return predicted_preferences

# 성능 평가 함수
def calculate_metrics(predicted, actual):
    accuracy = accuracy_score(actual, predicted) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(actual, predicted, average='weighted', zero_division=0)
    return accuracy, precision * 100, recall * 100, f1 * 100

# ResNet 특징 추출기 초기화 및 가중치 로드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor = ResNetFeatureExtractor(model_path).to(device)

# 엑셀 파일에서 데이터 추출 함수
def extract_preferences_from_excel(file_path):
    preferences_dict = {}
    preferences_df = pd.read_excel(file_path, engine='openpyxl')

    for idx, row in preferences_df.iterrows():
        respondent_id = row['Respondent ID']
        training_preferred = row['Training Style Preferred']
        training_not_preferred = row['Training Style Not Preferred']
        validation_preferred = row['Validation Style Preferred']
        validation_not_preferred = row['Validation Style Not Preferred']

        training_preferred_list = training_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(training_preferred) else []
        training_not_preferred_list = training_not_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(training_not_preferred) else []
        validation_preferred_list = validation_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(validation_preferred) else []
        validation_not_preferred_list = validation_not_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(validation_not_preferred) else []

        preferences_dict[respondent_id] = {
            'training_preferred': training_preferred_list,
            'training_not_preferred': training_not_preferred_list,
            'validation_preferred': validation_preferred_list,
            'validation_not_preferred': validation_not_preferred_list
        }

    return preferences_dict

# 엑셀 파일에서 선호 및 비선호 이미지 리스트 추출
preferences_dict = extract_preferences_from_excel(fixed_excel_file_path)

# Training Style Preferred 리스트 추출
train_style = []
for respondent_id, prefs in preferences_dict.items():
    train_style.extend(prefs['training_preferred'])

# Validation Style Preferred 및 Not Preferred 리스트 추출 및 스타일 선호도 할당
val_style = []
val_labels = []
for respondent_id, prefs in preferences_dict.items():
    val_style.extend(prefs['validation_preferred'])
    val_style.extend(prefs['validation_not_preferred'])
    val_labels.extend([1] * len(prefs['validation_preferred']))
    val_labels.extend([0] * len(prefs['validation_not_preferred']))

# 이미지 경로로 변환
train_image_paths = [os.path.join(train_image_dir, f"segmented_{img.strip()}") for img in train_style]
val_image_paths = [os.path.join(val_image_dir, f"segmented_{img.strip()}") for img in val_style]

# 학습 및 검증 이미지의 특징 벡터 추출
train_features = extract_features_for_dataset(train_image_paths, feature_extractor, device, log_missing_files=True)
val_features = extract_features_for_dataset(val_image_paths, feature_extractor, device, log_missing_files=True)

# 유사도 계산
cosine_similarity, euclidean_distance = calculate_similarity_scores(val_features, train_features)

# 최적의 alpha를 찾고 유사도를 결합하여 최종 점수 산출
optimal_alpha = find_optimal_alpha(cosine_similarity, euclidean_distance, [1] * len(train_features), val_labels)
combined_similarity_scores = combine_similarity_scores(cosine_similarity, euclidean_distance, optimal_alpha)

# 최적 threshold를 찾고 최종 예측
optimal_threshold = find_optimal_threshold(combined_similarity_scores, [1] * len(train_features), val_labels)
predicted_preferences = predict_style_preference(combined_similarity_scores, [1] * len(train_features), optimal_threshold)

# 성능 평가
accuracy, precision, recall, f1 = calculate_metrics(predicted_preferences, val_labels)

# 최종 결과 출력
print(f"Optimal Alpha: {optimal_alpha:.2f}")
print(f"Optimal Threshold: {optimal_threshold:.2f}")
print(f"Final Prediction Accuracy: {accuracy:.2f}%")
print(f"Final Precision: {precision:.2f}%")
print(f"Final Recall: {recall:.2f}%")
print(f"Final F1-Score: {f1:.2f}%")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-21-03a4d8eb5ce4>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

File not found: /content/drive/MyDrive/dataset/processed_segmentation_cleaned/segmented_T_15458_10_sportivecasual_M.jpg
File not found: /content/drive/MyDrive/dataset/processed_segmentation_cleaned/segmented_T_15453_10_sportivecasual_M.jpg
File not found: /content/drive/MyDrive/dataset/processed_segmentation_cleaned/segmented_T_15468_10_sportivecasual_M.jpg
File not found: /content/drive/MyDrive/dataset/processed_segmentation_cleaned/segmented_T_15477_10_sportivecasual_M.jpg
File not found: /content/drive/MyDrive/dataset/processed_segmentation_cleaned/segmented_T_15488_10_sportivecasual_M.jpg
File not found: /content/drive/MyDrive/dataset/processed_segmentation_cleaned/segmented_T_15467_10_sportivecasual_M.jpg


Extracting Features: 100%|██████████| 331/331 [00:02<00:00, 139.29it/s]


Missing files logged in 'missing_files_log.txt'


Extracting Features: 100%|██████████| 244/244 [00:01<00:00, 134.12it/s]


Missing files logged in 'missing_files_log.txt'
Optimal Alpha: 0.60
Optimal Threshold: 0.10
Final Prediction Accuracy: 78.69%
Final Precision: 82.84%
Final Recall: 78.69%
Final F1-Score: 76.56%


##최종 마감날 정리할때 2-2 표 내용이 바뀌면서 파일 누락이 생겨서 결과가 바뀌어서 밑에 부록으로 추가로 답니다.
## 디버깅할 시간이 없어서 이렇게 올려드리는점 양해 부탁드립니다.
## '/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/preferences0.xlsx'

## 원래 사용했던 preferences0.xlsx을 이용해서 다시 돌린 코드 결과물입니다.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

# 경로 설정
train_image_dir = '/content/drive/MyDrive/dataset/processed_segmentation_cleaned'
val_image_dir = '/content/drive/MyDrive/dataset/processed_segmentation_cleaned_for_val'
model_path = '/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/model_final5.pth'
excel_file_path = '/content/drive/MyDrive/데이터 크리에이터 캠프 최종 정리/김진/preferences0.xlsx'

# ResNet 특징 추출기 클래스 정의 (fc 레이어 제외)
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, model_path):
        super(ResNetFeatureExtractor, self).__init__()
        resnet = models.resnet18(pretrained=False)
        state_dict = torch.load(model_path)
        state_dict = {k: v for k, v in state_dict.items() if not k.startswith("fc.")}
        resnet.load_state_dict(state_dict, strict=False)
        self.features = nn.Sequential(*list(resnet.children())[:-2])

    def forward(self, x):
        return self.features(x)

# 이미지 전처리 파이프라인 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

# 데이터셋에서 이미지들의 특징 벡터를 추출하는 함수
def extract_features_for_dataset(image_paths, feature_extractor, device):
    features = []
    feature_extractor.eval()
    with torch.no_grad():
        for image_path in tqdm(image_paths, desc="Extracting Features"):
            if os.path.exists(image_path):
                image_tensor = load_and_preprocess_image(image_path, device)
                feature_vector = feature_extractor(image_tensor).squeeze().cpu().numpy()
                features.append(feature_vector)
            else:
                print(f"File not found: {image_path}")
    return np.array(features)

# 코사인 유사도 및 유클리디안 거리 계산 함수 (배치 처리 적용)
def calculate_similarity_scores(val_features, train_features, batch_size=100):
    cosine_similarity_list = []
    euclidean_distance_list = []

    val_features = torch.tensor(val_features).to(device)
    train_features = torch.tensor(train_features).to(device)
    val_features = val_features.view(val_features.size(0), -1)
    train_features = train_features.view(train_features.size(0), -1)

    with torch.no_grad():
        for i in range(0, len(val_features), batch_size):
            val_batch = val_features[i:i + batch_size]

            # 코사인 유사도 계산
            val_norm = val_batch / val_batch.norm(dim=1, keepdim=True)
            train_norm = train_features / train_features.norm(dim=1, keepdim=True)
            cosine_sim_batch = torch.mm(val_norm, train_norm.T).cpu().numpy()
            cosine_similarity_list.append(cosine_sim_batch)

            # 유클리디안 거리 계산
            val_batch_cpu = val_batch.cpu().numpy()
            train_features_cpu = train_features.cpu().numpy()
            euclidean_dist_batch = -np.linalg.norm(val_batch_cpu[:, None, :] - train_features_cpu[None, :, :], axis=2)
            euclidean_distance_list.append(euclidean_dist_batch)

    cosine_similarity = np.vstack(cosine_similarity_list)
    euclidean_distance = np.vstack(euclidean_distance_list)

    return cosine_similarity, euclidean_distance

# 두 유사도를 결합하는 함수
def combine_similarity_scores(cosine_similarity, euclidean_distance, alpha):
    return alpha * cosine_similarity + (1 - alpha) * euclidean_distance

# 최적 alpha 찾기
def find_optimal_alpha(cosine_similarity, euclidean_distance, train_preferences, val_labels):
    best_alpha, best_f1 = 0.5, 0
    for alpha in np.arange(0.1, 1.0, 0.1):
        combined_score = combine_similarity_scores(cosine_similarity, euclidean_distance, alpha)
        predicted_preferences = predict_style_preference(combined_score, train_preferences)
        _, _, _, f1 = calculate_metrics(predicted_preferences, val_labels)
        if f1 > best_f1:
            best_f1, best_alpha = f1, alpha
    return best_alpha

# 최적 threshold 찾기
def find_optimal_threshold(similarity_scores, train_preferences, val_labels):
    best_threshold, best_f1 = 0.5, 0
    for threshold in np.arange(0.1, 0.9, 0.01):
        predicted_preferences = predict_style_preference(similarity_scores, train_preferences, threshold)
        _, _, _, f1 = calculate_metrics(predicted_preferences, val_labels)
        if f1 > best_f1:
            best_f1, best_threshold = f1, threshold
    return best_threshold

# 스타일 선호도 예측
def predict_style_preference(similarity_scores, train_preferences, threshold=0.5):
    predicted_preferences = []
    train_len = len(train_preferences)  # train_preferences의 길이 확인

    for score_row in similarity_scores:
        max_similarity = np.max(score_row)
        # np.argmax(score_row)의 결과가 train_preferences 범위 내에 있는지 확인
        most_similar_index = np.argmax(score_row) if np.argmax(score_row) < train_len else train_len - 1
        predicted_label = train_preferences[most_similar_index] if max_similarity > threshold else 0
        predicted_preferences.append(predicted_label)

    return predicted_preferences

# 성능 평가 함수
def calculate_metrics(predicted, actual):
    accuracy = accuracy_score(actual, predicted) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(actual, predicted, average='weighted', zero_division=0)
    return accuracy, precision * 100, recall * 100, f1 * 100

# ResNet 특징 추출기 초기화 및 가중치 로드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor = ResNetFeatureExtractor(model_path).to(device)

# 엑셀 파일에서 데이터 추출 함수
def extract_preferences_from_excel(file_path):
    preferences_dict = {}
    preferences_df = pd.read_excel(file_path)

    for idx, row in preferences_df.iterrows():
        respondent_id = row['Respondent ID']
        training_preferred = row['Training Style Preferred']
        training_not_preferred = row['Training Style Not Preferred']
        validation_preferred = row['Validation Style Preferred']
        validation_not_preferred = row['Validation Style Not Preferred']

        training_preferred_list = training_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(training_preferred) else []
        training_not_preferred_list = training_not_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(training_not_preferred) else []
        validation_preferred_list = validation_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(validation_preferred) else []
        validation_not_preferred_list = validation_not_preferred.strip("[]").replace("'", "").split(", ") if pd.notna(validation_not_preferred) else []

        preferences_dict[respondent_id] = {
            'training_preferred': training_preferred_list,
            'training_not_preferred': training_not_preferred_list,
            'validation_preferred': validation_preferred_list,
            'validation_not_preferred': validation_not_preferred_list
        }

    return preferences_dict

# 엑셀 파일에서 선호 및 비선호 이미지 리스트 추출
preferences_dict = extract_preferences_from_excel(excel_file_path)

# Training Style Preferred 리스트 추출
train_style = []
for respondent_id, prefs in preferences_dict.items():
    train_style.extend(prefs['training_preferred'])

# Validation Style Preferred 및 Not Preferred 리스트 추출 및 스타일 선호도 할당
val_style = []
val_labels = []  # 검증 레이블 리스트 초기화
for respondent_id, prefs in preferences_dict.items():
    val_style.extend(prefs['validation_preferred'])
    val_style.extend(prefs['validation_not_preferred'])
    val_labels.extend([1] * len(prefs['validation_preferred']))
    val_labels.extend([0] * len(prefs['validation_not_preferred']))

# 이미지 경로로 변환
train_image_paths = [os.path.join(train_image_dir, f"segmented_{img.strip()}") for img in train_style]
val_image_paths = [os.path.join(val_image_dir, f"segmented_{img.strip()}") for img in val_style]

# 학습 및 검증 이미지의 특징 벡터 추출
train_features = extract_features_for_dataset(train_image_paths, feature_extractor, device)
val_features = extract_features_for_dataset(val_image_paths, feature_extractor, device)

# 유사도 계산
cosine_similarity, euclidean_distance = calculate_similarity_scores(val_features, train_features)

# 최적의 alpha를 찾고 유사도를 결합하여 최종 점수 산출
optimal_alpha = find_optimal_alpha(cosine_similarity, euclidean_distance, [1] * len(train_features), val_labels)
combined_similarity_scores = combine_similarity_scores(cosine_similarity, euclidean_distance, optimal_alpha)

# 최적 threshold를 찾고 최종 예측
optimal_threshold = find_optimal_threshold(combined_similarity_scores, [1] * len(train_features), val_labels)
predicted_preferences = predict_style_preference(combined_similarity_scores, [1] * len(train_features), optimal_threshold)

# 성능 평가
accuracy, precision, recall, f1 = calculate_metrics(predicted_preferences, val_labels)

# 최종 결과 출력
print(f"Optimal Alpha: {optimal_alpha:.2f}")
print(f"Optimal Threshold: {optimal_threshold:.2f}")
print(f"Final Prediction Accuracy: {accuracy:.2f}%")
print(f"Final Precision: {precision:.2f}%")
print(f"Final Recall: {recall:.2f}%")
print(f"Final F1-Score: {f1:.2f}%")


<ipython-input-28-bc3974a14a4c>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)
Extracting Features: 100%|██████████| 1102/1102 [00:07<

Optimal Alpha: 0.60
Optimal Threshold: 0.10
Final Prediction Accuracy: 67.88%
Final Precision: 74.04%
Final Recall: 67.88%
Final F1-Score: 61.82%
